# Sentiment analysis

## Analyses
1. ~~Simple keyword analysis~~
    * ~~tag cloud / bar chart visualisation~~
3. ~~Keyword analysis using word embeddings to conflate synonyms~~
    * Visualization of 2.?
4. ~~Sentiment analysis, e.g. pos/neg/neutral etc.~~
6. ~~Compare different cohorts, e.g. based on career stage, geography etc.  (if we have this data).~~
   * ~~Need to prioritise stratifications~~
5. Clustering of keywords, possibly using Carrot2 or similar?
7. Correlation analysis to see how responses are related
   * e.g. does working longer hours correlate with greater negative sentiment?


TODO

* ~~Load dataframe~~
* ~~Apply basic sentiment analysis to column 1~~
  * ~~do manual check~~
* ~~Apply SA to all columns~~
* ~~calculate average sentiment for each column~~
    * Experiment with variations on the VADER output, e.g. use other sentiment values (e.g. pos/neg)
* Programmatically iterate over all columns
* Test different SA algos
  * e.g. https://realpython.com/python-nltk-sentiment-analysis/
  * Can't really use ML without training data


## Prioritised stratifications
1. Role
1. Continent
2. Gender


## SA hypotheses:
* Role: more senior is more negative? *EVIDENCE FOUND*
* Gender: no difference expected *EVIDENCE FOUND*
* Geography: higher income more negative?
* Type of unit: Public sector more positive than private?

## Import data, clean & normalise

In [24]:
import pandas as pd

In [27]:
# read csv ignoring potential N/As
# df = pd.read_csv('Open Ended Questions-Table 1.csv', na_filter = False)
df = pd.read_csv('Data/Final data-Table 1.csv', na_filter = False)
# NB: Global Survey Data May24 - Final data was normalised prior to export to csv by removing 2nd header line and relabelling verbatim cols

In [28]:
df

,Respondent ID,Nationality,NAT,Year of birth,Gender,Marital status,Category 4: Marital status,In what country do you practice clinical embryology?,Category 1: Continent,Category 2: World bank classification,...,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Which best describes your professional morale and your feelings about the current state of clinical embryology profession?,Unnamed: 168,Unnamed: 169
0,19109318431,United Kingdom,186,1968,Female,Married or domestic partnership,Married or domestic partnership,United Kingdom,Europe and UK,High-income economies,...,,Paperwork burden,,,,,,3,,
1,19101225923,United Kingdom,186,1966,Male,"Single, never married","Single, never married",United Kingdom,Europe and UK,High-income economies,...,,Paperwork burden,Professional responsibility for errors,,,,,2,,
2,19096201682,Greece,67,1989,Female,Married or domestic partnership,Married or domestic partnership,Greece,Europe and UK,High-income economies,...,,,,Work dependency,Working hours/overtime,,,3,,
3,19095409638,United Kingdom,186,1990,Female,Married or domestic partnership,Married or domestic partnership,United Kingdom,Europe and UK,High-income economies,...,,,Professional responsibility for errors,,,,,4,,
4,19087353970,India,78,1998,Female,"Single, never married","Single, never married",India,Asian Countries,Lower-middle-income economies,...,Lack of professional recognition,,,,Working hours/overtime,,,5 - Very optimistic,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253,12834286411,South Africa,162,1978,Female,"Single, never married","Single, never married",South Africa,Sub-Saharan Africa (South Africa),Upper-middle-income economies,...,,,,,,,,4,,
1254,12834264931,South Africa,162,1987,Female,Married or domestic partnership,Married or domestic partnership,South Africa,Sub-Saharan Africa (South Africa),Upper-middle-income economies,...,Lack of professional recognition,,,Work dependency,,,,3,,
1255,12829654095,Peru,138,1962,Male,Divorced,Previously Married,Chile,South America,High-income economies,...,,,,,Working hours/overtime,,,5 - Very optimistic,,
1256,,,,,,,,,,,...,,,,,,,,,,


## Sentiment Analysis using NLTK

See https://www.datacamp.com/tutorial/text-analytics-beginners-nltk

In [319]:
# import libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [320]:
df['Q1a']

0                          Equality with clinical members
1                     Bullying by colleagues and managers
2             Working hours and too many responsibilities
3                                          burnout/stress
4                                             Recognizion
                              ...                        
1253    Respect from other departments for the complex...
1254                                        limited staff
1255                                             Training
1256                                                     
1257                                                     
Name: Q1a, Length: 1258, dtype: object

In [285]:
def analyse_cols(df, col1, col2):
    df = df.copy()  # Explicitly make a copy of the DataFrame
    df.loc[:, 'sentiment1'] = df.loc[:, col1].apply(get_compound_sentiment)
    df.loc[:, 'sentiment2'] = df.loc[:, col2].apply(get_compound_sentiment)
    with pd.option_context('display.max_colwidth', None):
        display(df[[col1, 'sentiment1', col2, 'sentiment2']])
    mean_sent = (df.loc[:, 'sentiment1'].mean() + df.loc[:, 'sentiment2'].mean())/2
    std_sent = (df.loc[:, 'sentiment1'].std() + df.loc[:, 'sentiment2'].std())/2 # a bit of a hack, should really square first
    print("Mean sentiment: {0:6.3f}".format(mean_sent))
    print("Std dev: {0:13.3f}".format(std_sent))
    return mean_sent

In [226]:
def analyse_col(df, col):
    df.loc[:, 'sentiment'] = df.loc[:, col].apply(get_compound_sentiment)
    with pd.option_context('display.max_colwidth', None):
        display(df[[col, 'sentiment']])
    mean_sent = (df.loc[:, 'sentiment'].mean())
    std_sent = (df.loc[:, 'sentiment'].std())
    print("Mean sentiment: {0:6.3f}".format(mean_sent))
    print("Std dev: {0:13.3f}".format(std_sent))
    return mean_sent

In [227]:
# initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [228]:
# compound get_sentiment function
def get_compound_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return scores['compound']

In [347]:
#  helper function to analyse sentiment across all 4 verbatims
def analyse_verbatims(index, in_df, out_df):
    out_df.iloc[index, res_df.columns.get_loc('Q1')] = analyse_cols(in_df, 'Q1a', 'Q1b')
    out_df.iloc[index, res_df.columns.get_loc('Q2')] = analyse_cols(in_df, 'Q2a', 'Q2b')
    out_df.iloc[index, res_df.columns.get_loc('Q3')] = analyse_cols(in_df, 'Q3a', 'Q3b')
    out_df.iloc[index, res_df.columns.get_loc('Q4')] = analyse_col(in_df, 'Q4')

## 1. Role

In [348]:
role_results = {'Category': ['Junior', 'Senior', 'Mgmt'],
        'Q1': [0.1, 0.2, 0.3],
        'Q2': [0.1, 0.2, 0.3],
        'Q3': [0.1, 0.2, 0.3],
        'Q4': [0.1, 0.2, 0.3]}
role_results_df = pd.DataFrame.from_dict(results)

In [349]:
role_results_df

,Category,Q1,Q2,Q3,Q4
0,Junior,0.1,0.1,0.1,0.1
1,Senior,0.2,0.2,0.2,0.2
2,Mgmt,0.3,0.3,0.3,0.3


### 1.1 Junior/trainee embryologist

In [355]:
junior_df = df[df["Category 3: Professional status"].isin(["Junior/trainee embryologist"])]
analyse_verbatims(0, junior_df, role_results_df)
role_results_df

,Q1a,sentiment1,Q1b,sentiment2
2,Working hours and too many responsibilities,0.0000,Salary,0.0000
4,Recognizion,0.0000,Trust,0.5106
6,General workplace politics,0.0000,Internal Pressure,-0.2960
7,Burnout due to working continously through weekends without breaks,0.0000,Difficulty in taking time off due to understaffed clinics,-0.3400
19,We are not being paid fairly (it's a specialized field),0.0000,Subjectivity when it comes to selecting/grading embryos,0.0000
...,...,...,...,...
1208,Too much paperwork,0.0000,Not enough focus on advancement,0.0000
1235,Understaffing,0.0000,Way too many hours worked,0.0000
1242,Increased workload,0.2732,Unpaid overtime and weekend work,0.0000
1243,The overtime work (sudden shifts),0.0000,Constant trouble shooting and little room for error,-0.6293


Mean sentiment: -0.047
Std dev:         0.242


,Q2a,sentiment1,Q2b,sentiment2
2,Access to training,0.0000,Certification,0.0000
4,Handson training,0.0000,Trust,0.5106
6,Undervalued,0.0000,Underpaid,0.0000
7,Lack of qualified embryologists - staff shortages,-0.4404,Underpayment for specialized skills,0.0000
19,Grading subjectivity,0.0000,Workplace stress,-0.4215
...,...,...,...,...
1208,Too few embryologists in general,0.0000,Lack of regulation / standardization,-0.3182
1235,Understaffing,0.0000,Way too many hours worked,0.0000
1242,Major responsibility,0.0000,Not always heard,0.0000
1243,Not paid according to the service done,0.0000,The high risk of burnout,-0.2732


Mean sentiment: -0.022
Std dev:         0.233


,Q3a,sentiment1,Q3b,sentiment2
2,Add reading and projects into basic workload not in leisure time,0.0000,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
4,Good laboratory training,0.4404,Troubleshooting,0.1779
6,Flexible hours to reduce stress,-0.2263,Shorter rotations in the Lab,0.0000
7,"Allow for more clinics to be accredited training units, thus increasing accessibility to science graduates and allowing more embryologists into the field",0.2263,"If an embryologist has worked on a weekend, allowing one day off during the week",0.0000
19,More recognition and a better payscale (minimally equivalent to the nurses),0.4877,More clinical involvement when it comes to deciding a patient's cycle/decision,0.0000
...,...,...,...,...
1208,"Standardize procedures/labeling, etc",0.0000,Provide more nurturing environment for juniors,0.5095
1235,Staff appropriately,0.0000,Cap number of hours worked,0.0772
1242,Overtime to a minimum,0.0000,Mindfullness of weekend work,0.0000
1243,Schedule stimulation protocols more accordingly,0.0000,Hire people to lessen administrative work,0.0000


Mean sentiment:  0.167
Std dev:         0.250


,Q4,sentiment
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
4,Be confident in the work i do Academically strong,0.7579
6,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
7,To be stable and happy while doing a job that I love.,0.8779
19,Laboratory manager/director,0.0000
...,...,...
1208,"PhD at EVMS, Lab Director (ABB HCLD)",0.0000
1235,To get out of embryology,0.0000
1242,I aspire to be a good technical and pratically skilled Embryologist with thorough knowledge and good patient communication and care skills.,0.8402
1243,To practice as a clinical embryologist in a private clinic,0.0000


Mean sentiment:  0.213
Std dev:         0.319


,Category,Q1,Q2,Q3,Q4
0,Junior,-0.046796,-0.022254,0.166524,0.212885
1,Senior,-0.047869,-0.035858,0.133322,0.194123
2,Mgmt,0.300000,0.300000,0.300000,0.300000


### 1.2 Senior embryologist

In [358]:
senior_df = df[df["Category 3: Professional status"].isin(["Senior embryologist"])]
analyse_verbatims(1, senior_df, role_results_df)
role_results_df

,Q1a,sentiment1,Q1b,sentiment2
0,Equality with clinical members,0.0000,Poor pay,-0.5423
1,Bullying by colleagues and managers,-0.5994,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
3,burnout/stress,0.0000,poor management,-0.4767
5,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024,No respect,0.2263
8,fertilization outcomes are laid on the embryologists,0.0000,pregnancies outcomes are laid on the embryologists,0.0000
...,...,...,...,...
1246,"not getting enough working materials, in ethiopia",0.0000,to get Professional maintenance is difficult in my area,-0.3612
1247,Too many cases per embryologist,0.0000,Not enough off-time,0.0000
1249,Working lots of weekends and public holidays,0.3818,workload,0.0000
1251,low salaries,-0.2732,weekend and holiday work,0.4019


Mean sentiment: -0.048
Std dev:         0.248


,Q2a,sentiment1,Q2b,sentiment2
0,Shortage of trained staff,-0.2500,Equality,0.0000
1,Stress of delivering high quality work without sufficient staff.,-0.4215,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
3,not enough highly trained staff,0.0000,politics,0.0000
5,Gossips of others which disturbs other Embryologist work,-0.6369,No respect,0.2263
8,overworking with minimum breaks (leave),0.0000,overall outcome of the patient pregnancy rates are laid on the embryologists,0.0000
...,...,...,...,...
1246,maintaining the quality control in the IVF lab,0.0000,"training of good embryology staffs,",0.4404
1247,Continuous skill development,0.0000,Not enough staff,0.0000
1249,workload,0.0000,stress,-0.4215
1251,low salaries,-0.2732,weekend and holiday work,0.4019


Mean sentiment: -0.036
Std dev:         0.233


,Q3a,sentiment1,Q3b,sentiment2
0,Improve number,0.4939,Improve pay,0.3612
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
3,better pay,0.3612,better CPD opportunities,0.6705
5,Reporting Manager should understand the situation,0.0000,Stress free atmosphere,0.1280
8,improve trust and provide fair team work to embryologist from the clinicians,0.8176,allow reasonable breaks to the embryologist for resting (leave),0.2263
...,...,...,...,...
1246,having good embryology training center,0.4404,support technicians financially to get training abroad,0.4019
1247,More staff to allow more personal time,0.2878,More opportunities to better our knowledge,0.7233
1249,increased wages (should get paid for weekends + public holidays),0.2732,employ more embryologists,0.0000
1251,increase salaries,0.3182,close over christmas,0.0000


Mean sentiment:  0.133
Std dev:         0.233


,Q4,sentiment
0,Continue,0.0000
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
3,FRCPath,0.0000
5,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
8,"to run an independant practice, be a partner and not an employee",0.0000
...,...,...
1246,be ART researcher.,0.0000
1247,To be the best embryologist I can be to the patients.,0.6369
1249,lab director,0.0000
1251,Dont make a big mistake,0.2584


Mean sentiment:  0.194
Std dev:         0.310


,Category,Q1,Q2,Q3,Q4
0,Junior,-0.046796,-0.022254,0.166524,0.212885
1,Senior,-0.047869,-0.035858,0.133322,0.194123
2,Mgmt,-0.048956,-0.045007,0.131119,0.199107


### 1.3 Management/director

In [357]:
mgmt_df = df[df["Category 3: Professional status"].isin(["Management/director"])]
analyse_verbatims(2, mgmt_df, role_results_df)
role_results_df

,Q1a,sentiment1,Q1b,sentiment2
9,lack of leadership,-0.3182,burnout,0.00
11,Time managment,0.0000,Long hours,0.00
14,Time spent at work,0.0000,Money for new quipment,0.00
15,Workload,0.0000,Underpaid,0.00
22,time management with patients/doctors,0.0000,including embryologists more in patient-treatment,0.00
...,...,...,...,...
1240,Lack of consideration by MDs,-0.3182,Understaffing,0.00
1248,sterility,0.0000,accuracy/prevention of errors,-0.34
1253,Respect from other departments for the complex work we do,0.4767,Financial compensation for overtime etc.,0.00
1254,limited staff,-0.2263,doctors,0.00


Mean sentiment: -0.049
Std dev:         0.228


,Q2a,sentiment1,Q2b,sentiment2
9,managment,0.0000,AI,0.0000
11,Lack of respect,0.2023,Having to make do with bare min equipment,0.0000
14,Low salary,-0.2732,Stress,-0.4215
15,Underpayment,0.0000,Lack of recognition,-0.3182
22,no mistakes are allowed,-0.5719,a lot of responsibility,0.0000
...,...,...,...,...
1240,Undervalued,0.0000,Underpaid,0.0000
1248,availability of positions,0.0000,working conditions,0.0000
1253,Financial participation: Owning shares,0.2960,lack of respect for the work we do by doctors,0.2023
1254,Salaries,0.0000,Limit work opportunities,0.3818


Mean sentiment: -0.045
Std dev:         0.241


,Q3a,sentiment1,Q3b,sentiment2
9,train managers to lead and not manage,0.0000,recognition,0.0000
11,Bigger budget for new equipment,0.0000,avoidance of working both days on weekends,-0.4019
14,More staff,0.0000,more off time,0.0000
15,Increase number of embryologists per lab,0.3818,Pay,-0.1027
22,no weekend work (better timing),-0.2960,knowledge that fertilization failure is not the fault of the embryologist,-0.2598
...,...,...,...,...
1240,Staffing guidelines/rules,0.0000,Educational guidelines and rules,0.0000
1248,Mon to Fri working hours,0.0000,8 hours per day,0.0000
1253,Pay them!,-0.1759,Invite financial incentives to the embryologists as well. e.g. Shares!,0.7574
1254,Better salaries,0.4404,Flexible working hours,0.2263


Mean sentiment:  0.131
Std dev:         0.217


,Q4,sentiment
9,have a happy team,0.5719
11,Done it,0.0000
14,Best success rate to promote our clinic,0.8885
15,Help struggling couples to have family of their own.,-0.0258
22,Lab-Leader,0.0000
...,...,...
1240,Retire with money in the bank,0.0000
1248,Building up of newly found laboratory in remote part of country,0.0000
1253,I reached the glass ceiling. All I can do is hope to study further and further and further.....,0.5106
1254,Laboratory director,0.0000


Mean sentiment:  0.199
Std dev:         0.291


,Category,Q1,Q2,Q3,Q4
0,Junior,-0.046796,-0.022254,0.166524,0.212885
1,Senior,-0.048956,-0.045007,0.131119,0.199107
2,Mgmt,-0.048956,-0.045007,0.131119,0.199107


### Show heatmap

In [359]:
import seaborn as sns
# cm = sns.light_palette("green", as_cmap=True)
# s = cont_results_df.style.background_gradient(cmap='viridis')
s = role_results_df.style.background_gradient()
s

,Category,Q1,Q2,Q3,Q4
0,Junior,-0.046796,-0.022254,0.166524,0.212885
1,Senior,-0.047869,-0.035858,0.133322,0.194123
2,Mgmt,-0.048956,-0.045007,0.131119,0.199107


## 2. Continent

In [265]:
cont_results = {'Continent': ['Asian', 'ANZ', 'EU', 'Mid East', 'NorthAm', 'NA (USA+Can)', 'SouthAm', 'SSAfrica', 'SSAfrica (SA)'],
        'Q1': [0.1, 0.2, 0.3,0.1, 0.2, 0.3, 0.1, 0.2, 0.3],
        'Q2': [0.1, 0.2, 0.3,0.1, 0.2, 0.3, 0.1, 0.2, 0.3],
        'Q3': [0.1, 0.2, 0.3,0.1, 0.2, 0.3, 0.1, 0.2, 0.3],
        'Q4': [0.1, 0.2, 0.3,0.1, 0.2, 0.3, 0.1, 0.2, 0.3]}
cont_results_df = pd.DataFrame.from_dict(cont_results)

In [266]:
cont_results_df

,Continent,Q1,Q2,Q3,Q4
0,Asian,0.1,0.1,0.1,0.1
1,ANZ,0.2,0.2,0.2,0.2
2,EU,0.3,0.3,0.3,0.3
3,Mid East,0.1,0.1,0.1,0.1
4,NorthAm,0.2,0.2,0.2,0.2
5,NA (USA+Can),0.3,0.3,0.3,0.3
6,SouthAm,0.1,0.1,0.1,0.1
7,SSAfrica,0.2,0.2,0.2,0.2
8,SSAfrica (SA),0.3,0.3,0.3,0.3


### 2.1 Asian Countries

In [360]:
asian_df = df[df["Category 1: Continent"].isin(["Asian Countries"])]
analyse_verbatims(0, asian_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
4,Recognizion,0.0000,Trust,0.5106
5,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024,No respect,0.2263
12,administration work,0.0000,not enough equipment,0.0000
25,Employees shortage,-0.2500,Untrained staff,0.0000
28,make no mistake,-0.5574,manage time properly,0.0000
...,...,...,...,...
1158,Results,0.0000,Waiting time,0.0000
1159,Day-5 blastocyst formation,0.0000,Thawed semen sample preparation,0.0000
1160,Interference of the clinician,0.0000,No credits for positives,0.5719
1161,Number of cases,0.0772,Quality Management,0.0000


Mean sentiment: -0.036
Std dev:         0.269


,Q2a,sentiment1,Q2b,sentiment2
4,Handson training,0.0000,Trust,0.5106
5,Gossips of others which disturbs other Embryologist work,-0.6369,No respect,0.2263
12,administration work,0.0000,lack of budget,-0.3182
25,Oocyte quality,0.0000,Implantation even after everything is good,0.4404
28,professional title evaluation,0.0000,paper publishing,0.0000
...,...,...,...,...
1158,NA,0.0000,NA,0.0000
1159,Thawed embryos results,0.0000,D-5 blastocyst formation or grading,0.0000
1160,Full time responsibility without any holidays,-0.2924,Paid less,0.0000
1161,Number of cases,0.0772,Quality Management,0.0000


Mean sentiment: -0.034
Std dev:         0.262


,Q3a,sentiment1,Q3b,sentiment2
4,Good laboratory training,0.4404,Troubleshooting,0.1779
5,Reporting Manager should understand the situation,0.0000,Stress free atmosphere,0.1280
12,a good system for administration work,0.4404,financial empowerment,0.0000
25,Should be adequate staff,0.2263,Time flexibility,0.3400
28,no,-0.2960,no,-0.2960
...,...,...,...,...
1158,NA,0.0000,NA,0.0000
1159,Avoid exposure of embryos as much you can,-0.2960,Assess embryos on day0- day1 and then on day 3 or 5 only,0.0000
1160,Definite working hours,0.2732,Proper trainings,0.0000
1161,Provide optimum number of embryologist depending on the number of cases,0.1531,Software to ease the data maintenance,0.3612


Mean sentiment:  0.152
Std dev:         0.245


,Q4,sentiment
4,Be confident in the work i do Academically strong,0.7579
5,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
12,to become scientific director with research project,0.0000
25,To find answers of failed implantation,-0.5106
28,lab director,0.0000
...,...,...
1158,Embryologist,0.0000
1159,Become a successful scientist,0.5859
1160,To help as many infertile couple as possible & to give them a healthy baby of their own.,0.6597
1161,To become a successful embryologist,0.5859


Mean sentiment:  0.285
Std dev:         0.321


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),-0.078252,-0.057447,0.134797,0.109998
6,SouthAm,0.008454,-0.031203,0.143214,0.167738
7,SSAfrica,-0.090921,-0.079295,0.140767,0.287852
8,SSAfrica (SA),-0.052014,-0.061212,0.172014,0.251500


### 2.2 ANZ

In [288]:
anz_df = df[df["Category 1: Continent"].isin(["Australia-New Zealand"])]
analyse_verbatims(1, anz_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
19,We are not being paid fairly (it's a specialized field),0.0000,Subjectivity when it comes to selecting/grading embryos,0.0000
49,Poor management,-0.4767,Stress caused by workload,-0.4215
126,underappreciation,0.0000,inner lab politics,0.0000
129,Workload,0.0000,Staffing,0.0000
130,Burn out from stress/lack of support,0.4019,Equipment supplies.,0.0000
...,...,...,...,...
930,Respect from clinicians,0.4767,Time,0.0000
1040,Salary,0.0000,Work flexibility,0.3400
1088,time,0.0000,poor training,-0.4767
1091,a lot of bad new calls falls on us and some patients havent been prepared to fail a cycle,-0.8256,a lot of patients just a have no idea what role we actually play in the cycle,0.0516


Mean sentiment: -0.101
Std dev:         0.234


,Q2a,sentiment1,Q2b,sentiment2
19,Grading subjectivity,0.0000,Workplace stress,-0.4215
49,Poor management,-0.4767,Bullying and nepotism,-0.5994
126,stress,-0.4215,multiple tasking,0.0000
129,Workload,0.0000,Staffing,0.0000
130,Appropriate renumeration -Recognition of the pressures of the role.,-0.3182,Not enough acknowledgment that experience is just as relevant (if not more so) than a grad degree.,0.0000
...,...,...,...,...
930,Respect from clinicians,0.4767,Too many graduates who are unskilled and unsuited to the profession,0.0000
1040,Keeping trained staff,0.0000,Salary,0.0000
1088,career advancement,0.0000,salary,0.0000
1091,low pay for literally making life,-0.3612,expected to perform miracles and getting slammed when it doesnt happen,0.0000


Mean sentiment: -0.063
Std dev:         0.207


,Q3a,sentiment1,Q3b,sentiment2
19,More recognition and a better payscale (minimally equivalent to the nurses),0.4877,More clinical involvement when it comes to deciding a patient's cycle/decision,0.0000
49,National pay and conditions standards,-0.1027,Doctor owned clinics care more about patients and staff. They should be supported more,0.6705
126,adding admins to the team to take care of the paper work and reports and let the embryologists focus on lab work,0.4939,clear career/training path,0.3818
129,More practical training programs,0.0000,Greater advertisement at unis about the embryology career,0.3612
130,Better/reliable equipment and having stock available.,0.0000,Better mental health support and recognition - better renumeration,0.8176
...,...,...,...,...
930,Improved staffing ratios,0.4767,Better access to AFFORDABLE training and further education,0.4404
1040,EA agreements at all IVF Clinics,0.2732,Set days for OPU's to limit Saturday and Sunday work hours,0.0000
1088,pay them well,0.1779,professional association,0.0000
1091,higher pay,-0.1027,more public education about IVF,0.0000


Mean sentiment:  0.133
Std dev:         0.222


,Q4,sentiment
19,Laboratory manager/director,0.0000
49,Retirement,0.0000
126,Money :),0.4588
129,Be a successful Embryologist,0.5859
130,Keep working,0.0000
...,...,...
930,To win the lotto!!,0.6580
1040,To stay working in the IVF field,0.0000
1088,mentoring pass on my knowledge establish a large research footprint,0.0000
1091,"to become a fully qualified embryologist, potentially lab manager or higher later down the track.",0.0000


Mean sentiment:  0.181
Std dev:         0.306


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,0.300000,0.300000,0.300000,0.300000
3,Mid East,0.100000,0.100000,0.100000,0.100000
4,NorthAm,0.200000,0.200000,0.200000,0.200000
5,NA (USA+Can),0.300000,0.300000,0.300000,0.300000
6,SouthAm,0.100000,0.100000,0.100000,0.100000
7,SSAfrica,0.200000,0.200000,0.200000,0.200000
8,SSAfrica (SA),0.300000,0.300000,0.300000,0.300000


### 2.3 EU

In [289]:
eu_df = df[df["Category 1: Continent"].isin(["Europe and UK"])]
analyse_verbatims(2, eu_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
0,Equality with clinical members,0.0000,Poor pay,-0.5423
1,Bullying by colleagues and managers,-0.5994,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
2,Working hours and too many responsibilities,0.0000,Salary,0.0000
3,burnout/stress,0.0000,poor management,-0.4767
9,lack of leadership,-0.3182,burnout,0.0000
...,...,...,...,...
1220,Stress,-0.4215,Workload,0.0000
1227,Clinical work load per embryologist,0.0000,Difficulty to voice suggestions for improvement,0.1531
1237,oocyte vitrification,0.0000,more sophisticated witnessing - software etc.,0.5984
1238,Quality control,0.0000,Work pressure,-0.2960


Mean sentiment: -0.039
Std dev:         0.235


,Q2a,sentiment1,Q2b,sentiment2
0,Shortage of trained staff,-0.2500,Equality,0.0000
1,Stress of delivering high quality work without sufficient staff.,-0.4215,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
2,Access to training,0.0000,Certification,0.0000
3,not enough highly trained staff,0.0000,politics,0.0000
9,managment,0.0000,AI,0.0000
...,...,...,...,...
1220,Globalisation / commercialisation,0.0000,Reliance on technicians rather than embryologists,0.0000
1227,Work/personal life balance,0.0000,Poor envolvment on R&D,-0.4767
1237,AI for embryo - selection,0.0000,NiPGT,0.0000
1238,Quality management control,0.0000,Quality management assurance,0.3400


Mean sentiment: -0.011
Std dev:         0.222


,Q3a,sentiment1,Q3b,sentiment2
0,Improve number,0.4939,Improve pay,0.3612
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
2,Add reading and projects into basic workload not in leisure time,0.0000,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
3,better pay,0.3612,better CPD opportunities,0.6705
9,train managers to lead and not manage,0.0000,recognition,0.0000
...,...,...,...,...
1220,Workload per embryologist guidelines,0.0000,Daylight,0.0000
1227,"Limit per embryologist numbers of time consuming procedures (biopsy, Tesa/pesa, egg freeze)",0.0000,Provide help admin/technicians/software,0.4019
1237,IVF lab facility,0.0000,good collective,0.4404
1238,Administrative simplification,0.0000,Clinic-laboratory communication,0.0000


Mean sentiment:  0.117
Std dev:         0.222


,Q4,sentiment
0,Continue,0.0000
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
3,FRCPath,0.0000
9,have a happy team,0.5719
...,...,...
1220,Survive!,0.0000
1227,Laboratory manager,0.0000
1237,I am satisfied :-),0.6249
1238,Senior clinical embryologist,0.0000


Mean sentiment:  0.169
Std dev:         0.292


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,0.100000,0.100000,0.100000,0.100000
4,NorthAm,0.200000,0.200000,0.200000,0.200000
5,NA (USA+Can),0.300000,0.300000,0.300000,0.300000
6,SouthAm,0.100000,0.100000,0.100000,0.100000
7,SSAfrica,0.200000,0.200000,0.200000,0.200000
8,SSAfrica (SA),0.300000,0.300000,0.300000,0.300000


### 2.4 Middle East

In [291]:
me_df = df[df["Category 1: Continent"].isin(["Middle Eastern Countries"])]
analyse_verbatims(3, me_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
17,Oocyte quality,0.0000,Machine fixation,0.000
26,Stress induced work environment,-0.4215,No support to access clinical embryology knowledge,0.128
27,Recognition and sallary,0.0000,Supply of devices and consumables,0.000
33,BURN OUT,0.0000,Doctors always blame embryologist,-0.340
35,emotional stress,-0.2960,patients/doctors demands,0.000
...,...,...,...,...
1077,Load of cases to number of embryologists working,0.0772,"Hence, time",0.000
1079,equipment,0.0000,training,0.000
1093,No equality in work because we don't have specific governmental rules for Embryologists.,-0.2960,No scientific support in all ivf updates overall,0.128
1094,Timing between work and studying,0.0000,Work load,0.000


Mean sentiment: -0.019
Std dev:         0.224


,Q2a,sentiment1,Q2b,sentiment2
17,Embryo quality,0.0000,Pregnancy rate,0.0000
26,Lack of organized regulations community,-0.3182,AI,0.0000
27,Training,0.0000,Recognition and sallary,0.0000
33,NO WORK-LIFE BALANCE,-0.2960,stress,-0.4215
35,emotional stress,-0.2960,afraid to make a mistake,-0.3400
...,...,...,...,...
1077,Physicians,0.0000,Regualtions that protect the embyologist,0.3818
1079,training,0.0000,equipmet,0.0000
1093,Financial issues,0.0000,Follow up updates,0.0000
1094,Work load,0.0000,F,0.0000


Mean sentiment: -0.038
Std dev:         0.266


,Q3a,sentiment1,Q3b,sentiment2
17,Control case number by day,0.0772,Checking working machine regularly,0.0000
26,Union for embryologist,0.0000,Increase scientific support through worksfop and continuous education,0.6124
27,Sponsoring for attending events,0.0000,Sponsoring for training,0.0000
33,Hire staff according to workload,0.0000,Give embryologists space to work,0.0000
35,emotional support,0.5106,financial benefits,0.3818
...,...,...,...,...
1077,Regualtions,0.0000,Support for small private clinics,0.4019
1079,online cource,0.0000,academic coure,0.0000
1093,More employee's to divid the tasks well.,0.2732,More skills more meetings.. More scientific papers must published for each ivf clinic,0.0000
1094,Training,0.0000,Superving,0.0000


Mean sentiment:  0.129
Std dev:         0.224


,Q4,sentiment
17,Good embryo quality,0.4404
26,Partnership in ivf center,0.0000
27,Highest success rates,0.5719
33,To be a Quality Manager and audit labs,0.0000
35,lab director,0.0000
...,...,...
1077,Shift to women’s counseling,0.0000
1079,scientist,0.0000
1093,My ambition is to increase my self esteem when I show sterile couples having babies by developing my self scientifically in order to treat difficult sterile cases.,0.3612
1094,Be professional,0.0000


Mean sentiment:  0.204
Std dev:         0.289


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,0.200000,0.200000,0.200000,0.200000
5,NA (USA+Can),0.300000,0.300000,0.300000,0.300000
6,SouthAm,0.100000,0.100000,0.100000,0.100000
7,SSAfrica,0.200000,0.200000,0.200000,0.200000
8,SSAfrica (SA),0.300000,0.300000,0.300000,0.300000


### 2.5 North America

In [292]:
na_df = df[df["Category 1: Continent"].isin(["North America"])]
analyse_verbatims(4, na_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
61,Excessive overtime,0.0000,Career progression,0.0000
67,Burn-out,0.0000,poor leadership,-0.4767
89,Long hours,0.0000,Lack of pay,-0.4019
194,Under staff,0.0000,Career growth,0.3818
203,Stress due to high workload,-0.4215,Physical challenges from long periods at microscope,0.0772
204,Recognition of the role of embryologists,0.0000,Career advancement,0.0000
250,Overwork,0.0000,Professional burnout,0.0000
291,Updating dated equipment,0.0000,Staffing,0.0000
321,staffing,0.0000,workload,0.0000
413,WORKING 100% ERROR FREE EVERY DAY,0.1531,DISINTEREST IN FEEDBACK BY OTHER GROUPS,0.0000


Mean sentiment: -0.051
Std dev:         0.210


,Q2a,sentiment1,Q2b,sentiment2
61,Overworked,0.0000,Underpaid,0.0000
67,low compensation,-0.2732,burn-out,0.0000
89,Lack of new recruits,-0.3182,Low pay for hours worked,-0.3612
194,Recognition,0.0000,No platform to share knowledge,0.0000
203,Lack of embryologists,-0.3182,High burn out,0.0000
204,Regulation,0.0000,Standardization,0.0000
250,Professional burnout,0.0000,Lack of career opportunities li,0.0772
291,Dated equipment,0.0000,Staffing,0.0000
321,lack of qualified new staff,-0.3182,uneven workload,0.0000
413,LACK OF UNDERSTANDING OF THE JOB OUTSIDE OF THE LAB,-0.3182,MAINTAINING AN ADEQUATE STAFFING LEVEL AT ALL TIMES,0.2263


Mean sentiment: -0.059
Std dev:         0.209


,Q3a,sentiment1,Q3b,sentiment2
61,More embryologists,0.0000,Better pay,0.3612
67,"better compensation in terms of salary, profit sharing",0.8225,"people management and leadership trainings for the laboratory supervisors, managers and director",0.0000
89,Higher pay,-0.1027,More people,0.0000
194,Adequate staff in the lab,0.2263,A fixed timeline to get trained & gain experience in order to go higher up in the hierarchy,0.5267
203,Better working environment in lab,0.4404,Improved communication between departments including embryology,0.4767
204,Regulation of practice,0.0000,Standardization,0.0000
250,Support life/work balance,0.4019,Engage for professional development,0.3400
291,Budget for new equipment,0.0000,Staffing of personnel to alleviate workload - admin. lab assistant,0.0000
321,more staff,0.0000,more control over scheduling of cycles,0.0000
413,HAVING A BIGGER VOICE WITHIN THE INSTITUTION/PROFESSION,0.0000,"A BETTER UNDERSTAINDING OF THE JOB BY PHYSICIANS/BUSINESS MANAGERS, ETC.",0.4404


Mean sentiment:  0.179
Std dev:         0.243


,Q4,sentiment
61,Be the best I can at the job I do to provide patients with best possible care,0.9118
67,attain my PhD. become a consultant Reproductive scientist. dive into teaching. establish an embryology training center.,0.0000
89,Retirement,0.0000
194,Head a IVF unit & apply latest advancements in a clinical setting to hep smoothen the path of parenthood.,0.0000
203,Supervisor,0.0000
204,Contribute to the advancement of new technologies to improve ART outcomes through clinical research,0.4404
250,Career change,0.0000
291,Continuing education,0.0000
321,to retire,0.0000
413,N/A,0.0000


Mean sentiment:  0.114
Std dev:         0.270


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),0.300000,0.300000,0.300000,0.300000
6,SouthAm,0.100000,0.100000,0.100000,0.100000
7,SSAfrica,0.200000,0.200000,0.200000,0.200000
8,SSAfrica (SA),0.300000,0.300000,0.300000,0.300000


### 2.6 North America (USA+Canada)

In [293]:
nausa_df = df[df["Category 1: Continent"].isin(["North America (USA and Canada)"])]
analyse_verbatims(5, nausa_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
56,Lack of staff,-0.3182,Burnout derived from lack of staff,-0.3182
91,Overworked,0.0000,Understaffed,0.0000
119,Too much work,0.0000,Short staffed,0.0000
122,Short staffing,0.0000,Work load,0.0000
154,understaff,0.0000,lack of training,-0.3182
...,...,...,...,...
1233,Burnout,0.0000,Understaffing,0.0000
1234,Fatigue,-0.2500,Lack of recognition,-0.3182
1235,Understaffing,0.0000,Way too many hours worked,0.0000
1236,Patient workload,0.0000,Work stress,-0.4215


Mean sentiment: -0.078
Std dev:         0.220


,Q2a,sentiment1,Q2b,sentiment2
56,Stress,-0.4215,Burnout,0.0000
91,Overworked,0.0000,Understaffed,0.0000
119,Not enough staff for workload,0.0000,Underpaid,0.0000
122,Short staffing,0.0000,Pay,-0.1027
154,furthering careers,0.0000,advanced age in the lab,0.2500
...,...,...,...,...
1233,Demanding schedules,-0.2263,Lack of trained workforce,-0.3182
1234,Stress,-0.4215,Lack of support,0.1027
1235,Understaffing,0.0000,Way too many hours worked,0.0000
1236,Work stress,-0.4215,Lack of time off,-0.3182


Mean sentiment: -0.057
Std dev:         0.217


,Q3a,sentiment1,Q3b,sentiment2
56,"It really comes down to the Director of the center, but to hire more staff",0.0000,Provide flexible schedules for embryologist as well as a planned out vacation system for lab staff,0.4588
91,Rotate daily duties,0.0000,Use Per Diem help when needed,0.4019
119,Hire more staff,0.0000,Provide proper training,0.0000
122,More support from management in regards to work load and staffing,0.4576,Increased pay,0.1779
154,...,0.0000,...,0.0000
...,...,...,...,...
1233,Proper staffing,0.0000,Proper compensation,0.0000
1234,Better support from doctors,0.6808,Work fewer weekends,0.0000
1235,Staff appropriately,0.0000,Cap number of hours worked,0.0772
1236,Hire more personel,0.0000,Better benifits,0.4404


Mean sentiment:  0.135
Std dev:         0.218


,Q4,sentiment
56,Become IVF lab director and author publications,0.0000
91,Move to an IVF Industry,0.0000
119,Stay employed at current employer until retirement,0.0000
122,Maintain a career as a successful embryologist,0.5859
154,...,0.0000
...,...,...
1233,Have a position that uses my knowledge but has a flexible schedule. Possibly training or sales position,0.3291
1234,I have reached senior status,0.1027
1235,To get out of embryology,0.0000
1236,To retire,0.0000


Mean sentiment:  0.110
Std dev:         0.277


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),-0.078252,-0.057447,0.134797,0.109998
6,SouthAm,0.100000,0.100000,0.100000,0.100000
7,SSAfrica,0.200000,0.200000,0.200000,0.200000
8,SSAfrica (SA),0.300000,0.300000,0.300000,0.300000


### 2.7 South America

In [297]:
sa_df = df[df["Category 1: Continent"].isin(["South America"])]
analyse_verbatims(6, sa_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
20,Biopsy,0.0000,Icsi,0.0000
23,Pression,0.0000,Rare free days,0.5106
24,Daily work hours and weekends,0.0000,Salary,0.0000
36,A good salary,0.4404,Overtime in the working day,0.0000
45,TRUST,0.5106,Long work routines,0.0000
...,...,...,...,...
1048,laboratory quality control and service quality improvements,0.3182,work together with the medical group,0.0000
1050,Wage,0.0000,Participación un congress,0.0000
1059,Training,0.0000,Place to work,0.0000
1241,Access to knowledge about QC / QA and the tools to apply them,0.0000,Learning embryo biopsy,0.0000


Mean sentiment:  0.008
Std dev:         0.201


,Q2a,sentiment1,Q2b,sentiment2
20,People,0.000,Dinâmic,0.0000
23,Recognition,0.000,Salary,0.0000
24,Higher recognition,0.000,Higher salaries,0.0000
36,"Undervalued profession, regarding the responsibility it implies.",0.000,Few job offers in your country.,0.0000
45,Sacrifice,0.000,Family,0.0000
...,...,...,...,...
1048,the high expectations that patients and doctors have about the techniques we perform,0.000,The pressure of rapid technological changes: discerning the value of them,0.0516
1050,Wage,0.000,Equality,0.0000
1059,.,0.000,.,0.0000
1241,Earn a salary according to the profession,0.000,Professional recognition,0.0000


Mean sentiment: -0.031
Std dev:         0.202


,Q3a,sentiment1,Q3b,sentiment2
20,Psycology,0.0000,Dinâmic group,0.0000
23,Base and high salary,0.0000,Free weekends,0.5106
24,Higher salaries,0.0000,Flexibility In the laboratory,0.3400
36,establish a real commitment and communication between doctors and embryologists in the workplace.,0.3818,involving the embryologist every day to participate in more activities outside the in vitro laboratory,0.0000
45,that we can attend more congresses and with help,0.4019,improve salary,0.4404
...,...,...,...,...
1048,no restrictions on the use of disposable materials and media,-0.2960,share responsibilities between physians and embryologists,0.2960
1050,Increases wages,0.0000,Continuous training programs,0.0000
1059,Salary,0.0000,Training,0.0000
1241,Improve access to training,0.4404,Improve wages,0.4404


Mean sentiment:  0.143
Std dev:         0.231


,Q4,sentiment
20,becoming an embryologist always better than before,0.4404
23,Be a recognized embryologist in the world,0.0000
24,Higher salary,0.0000
36,Train more embryologists.,0.0000
45,"improve every day, new techniques and equipment so that our work is optimal",0.6597
...,...,...
1048,Give the best service,0.6369
1050,To be internacional Consultant,0.0000
1059,.,0.0000
1241,Have the best reproductive results in my country,0.6369


Mean sentiment:  0.168
Std dev:         0.279


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),-0.078252,-0.057447,0.134797,0.109998
6,SouthAm,0.008454,-0.031203,0.143214,0.167738
7,SSAfrica,-0.090921,-0.079295,0.140767,0.287852
8,SSAfrica (SA),0.300000,0.300000,0.300000,0.300000


### 2.8 Sub-Saharan Africa

In [361]:
ssa_df = df[df["Category 1: Continent"].isin(["Sub-Saharan Africa"])]
analyse_verbatims(7, ssa_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
53,Little or no day off alloted,-0.2315,Employer doubting one's working ability,-0.0258
60,Burnout,0.0000,Lack of New Technologies,-0.3182
125,increase in work load,0.3182,lack of staffs,-0.3182
127,Undermining the competence level of the embryologist by the clinician,-0.3612,Embryologist are not involved in decisions making with regards to the client preparation,0.0000
128,Work overload,-0.3612,Low salary,-0.2732
165,less equipment,0.0000,low salary,-0.2732
168,Retraining are expensive,0.0000,Sponsorship for conference and scientific program not available,0.0000
181,give best results in every month,0.6369,to convene clinicians,0.0000
293,Work load,0.0000,Renumeration,0.0000
330,Interference by clinicians in lab issues,0.0000,None,0.0000


Mean sentiment: -0.091
Std dev:         0.305


,Q2a,sentiment1,Q2b,sentiment2
53,Unfavourable working conditions,0.0000,Short-staffed,0.0000
60,Burnout,0.0000,Slow adaptation to new technologies,0.0000
125,lack of more training center for study,-0.3182,increase in work load,0.3182
127,We have no regulations body for embryologists in my count,-0.2960,The gynecologist doesn’t respect the embryo since all the ivf centers are private owned,0.4767
128,Work overload,-0.3612,Insufficient vacation time,0.0000
165,no scholarship for student in the field to further,-0.2960,no access to research papers,-0.2960
168,Knowledge updates are expensive,0.0000,Programs are expensive to study further,0.0000
181,shortage of skilled professionals,-0.2500,lack of proper hands on training,-0.3182
293,Much work load,0.0000,Poor Welfare,-0.4767
330,None,0.0000,None,0.0000


Mean sentiment: -0.079
Std dev:         0.225


,Q3a,sentiment1,Q3b,sentiment2
53,Allot days for the embryologist to rest,0.0000,Batch cycles as opposed to daily/weekly treatments,0.0000
60,More embryologist,0.0000,Better technology,0.4404
125,provision of more training centers for embryology study,0.0000,introduction of more staffs,0.0000
127,The involvement in the day activities with regards to the patient preparation for the procedure,0.0000,Regular training to improve the efficiency and effectiveness of the embryologist in the country,0.6597
128,Proper planning of cases and effective communication.,0.4767,Good pay for work done,0.3612
165,give fundings to less priveledge embryologists for studies and research,0.0000,make more journals accessible,0.0000
168,Sponsored training and retraining,0.0000,Research sponsored,0.0000
181,clinicians should take suggestions before take final decision,0.0000,by recognising role of embryologist in IVF treatment,0.0000
293,Employed more staff,0.0000,Better renumeration,0.4404
330,Embryologists decisions should be respected by clinicians,0.4767,None,0.0000


Mean sentiment:  0.141
Std dev:         0.226


,Q4,sentiment
53,To help couples who struggle with infertility become parents and bring joy into their homes,0.6369
60,To become a Chief Scientific Director of a chain,0.0000
125,TO become a clinical embryologist in time to come,0.0000
127,An outstanding fertility specialist,0.6124
128,Become the laboratory director and quality control director of an embryology laboratory. Be a National decision maker in ART.,0.0000
165,to be able to improve human existence with assisted reproductive procedures,0.4404
168,To provide first class services to patients,0.0000
181,to maintain high success rates,0.5719
293,To become a professional clinical embryologist,0.0000
330,To assist in getting at least 1000 babies being born from IVF/ICSI procedures,0.0000


Mean sentiment:  0.288
Std dev:         0.301


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),-0.078252,-0.057447,0.134797,0.109998
6,SouthAm,0.008454,-0.031203,0.143214,0.167738
7,SSAfrica,-0.090921,-0.079295,0.140767,0.287852
8,SSAfrica (SA),-0.052014,-0.061212,0.172014,0.251500


### 2.9 Sub-Saharan Africa (South Africa)

In [362]:
# Next line returns an empty set, needs checking. As a workaround we can filter using different field
# ssasa_df = df[df["Category 1: Continent"].isin(["Sub-Saharan Africa (South Africa)"])]
ssasa_df = df[df["In what country do you practice clinical embryology?"].isin(["South Africa"])]
analyse_verbatims(8, ssasa_df, cont_results_df)
cont_results_df

,Q1a,sentiment1,Q1b,sentiment2
6,General workplace politics,0.0000,Internal Pressure,-0.2960
7,Burnout due to working continously through weekends without breaks,0.0000,Difficulty in taking time off due to understaffed clinics,-0.3400
8,fertilization outcomes are laid on the embryologists,0.0000,pregnancies outcomes are laid on the embryologists,0.0000
10,Recognition of our contribution in the ART field,0.0000,Workload,0.0000
11,Time managment,0.0000,Long hours,0.0000
...,...,...,...,...
1250,Low Salaries,-0.2732,Working weekends,0.0000
1251,low salaries,-0.2732,weekend and holiday work,0.4019
1252,Low salaries,-0.2732,Weekend and holiday work,0.4019
1253,Respect from other departments for the complex work we do,0.4767,Financial compensation for overtime etc.,0.0000


Mean sentiment: -0.052
Std dev:         0.240


,Q2a,sentiment1,Q2b,sentiment2
6,Undervalued,0.0000,Underpaid,0.0000
7,Lack of qualified embryologists - staff shortages,-0.4404,Underpayment for specialized skills,0.0000
8,overworking with minimum breaks (leave),0.0000,overall outcome of the patient pregnancy rates are laid on the embryologists,0.0000
10,Inferior treatment by Clinicians,-0.4019,Workload,0.0000
11,Lack of respect,0.2023,Having to make do with bare min equipment,0.0000
...,...,...,...,...
1250,Not closing over December,0.0000,Low Salaries,-0.2732
1251,low salaries,-0.2732,weekend and holiday work,0.4019
1252,Low salary,-0.2732,Weekend and holiday work,0.4019
1253,Financial participation: Owning shares,0.2960,lack of respect for the work we do by doctors,0.2023


Mean sentiment: -0.061
Std dev:         0.261


,Q3a,sentiment1,Q3b,sentiment2
6,Flexible hours to reduce stress,-0.2263,Shorter rotations in the Lab,0.0000
7,"Allow for more clinics to be accredited training units, thus increasing accessibility to science graduates and allowing more embryologists into the field",0.2263,"If an embryologist has worked on a weekend, allowing one day off during the week",0.0000
8,improve trust and provide fair team work to embryologist from the clinicians,0.8176,allow reasonable breaks to the embryologist for resting (leave),0.2263
10,Awareness by Clinic owners,0.0000,Educate us how to successfully get what we are asking for,0.4939
11,Bigger budget for new equipment,0.0000,avoidance of working both days on weekends,-0.4019
...,...,...,...,...
1250,Higher salaries,0.0000,Closing over December,0.0000
1251,increase salaries,0.3182,close over christmas,0.0000
1252,Higher salaries,0.0000,Close over Christmas holidays,0.3818
1253,Pay them!,-0.1759,Invite financial incentives to the embryologists as well. e.g. Shares!,0.7574


Mean sentiment:  0.172
Std dev:         0.260


,Q4,sentiment
6,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
7,To be stable and happy while doing a job that I love.,0.8779
8,"to run an independant practice, be a partner and not an employee",0.0000
10,This is more of an aspiration - I want my unit to be the best in the country,0.6705
11,Done it,0.0000
...,...,...
1250,To complete my masters and become the best practical embryologist to my abilities.,0.7351
1251,Dont make a big mistake,0.2584
1252,Make babies and make people happy.,0.5719
1253,I reached the glass ceiling. All I can do is hope to study further and further and further.....,0.5106


Mean sentiment:  0.252
Std dev:         0.347


,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),-0.078252,-0.057447,0.134797,0.109998
6,SouthAm,0.008454,-0.031203,0.143214,0.167738
7,SSAfrica,-0.090921,-0.079295,0.140767,0.287852
8,SSAfrica (SA),-0.052014,-0.061212,0.172014,0.251500


### Show heatmap

In [363]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)
# s = cont_results_df.style.background_gradient(cmap='viridis')
s = cont_results_df.style.background_gradient()
s

,Continent,Q1,Q2,Q3,Q4
0,Asian,-0.036344,-0.033745,0.151976,0.284547
1,ANZ,-0.101317,-0.062772,0.133450,0.181388
2,EU,-0.038519,-0.010884,0.117218,0.169275
3,Mid East,-0.018551,-0.038200,0.128752,0.203658
4,NorthAm,-0.051093,-0.058659,0.179170,0.113900
5,NA (USA+Can),-0.078252,-0.057447,0.134797,0.109998
6,SouthAm,0.008454,-0.031203,0.143214,0.167738
7,SSAfrica,-0.090921,-0.079295,0.140767,0.287852
8,SSAfrica (SA),-0.052014,-0.061212,0.172014,0.251500


In [11]:
df

,What are the TWO major challenges faced by embryologists in the workplace?,Unnamed: 1,"What are the TWO major challenges faced by the embryology profession, in general?",Unnamed: 3,Provide TWO suggestions to improve embryologists' working conditions,Unnamed: 5,What is your career goal?
0,1,2,1,2,1,2,Open-Ended Response
1,Equality with clinical members,Poor pay,Shortage of trained staff,Equality,Improve number,Improve pay,Continue
2,Working hours and too many responsibilities,Salary,Access to training,Certification,Add reading and projects into basic workload n...,Understand for how many hours a brain can be f...,Get my ESHRE certification Publish paper thro...
3,burnout/stress,poor management,not enough highly trained staff,politics,better pay,better CPD opportunities,FRCPath
4,Recognizion,Trust,Handson training,Trust,Good laboratory training,Troubleshooting,Be confident in the work i do Academically st...
...,...,...,...,...,...,...,...
853,Low Salaries,Working weekends,Not closing over December,Low Salaries,Higher salaries,Closing over December,To complete my masters and become the best pra...
854,low salaries,weekend and holiday work,low salaries,weekend and holiday work,increase salaries,close over christmas,Dont make a big mistake
855,Low salaries,Weekend and holiday work,Low salary,Weekend and holiday work,Higher salaries,Close over Christmas holidays,Make babies and make people happy.
856,Respect from other departments for the complex...,Financial compensation for overtime etc.,Financial participation: Owning shares,lack of respect for the work we do by doctors,Pay them!,Invite financial incentives to the embryologis...,I reached the glass ceiling. All I can do is h...


In [12]:
# create a dict for the short and log column names
long_col_names = {'Q1a': df.columns[0], 'Q1b': df.columns[0], 'Q2a': df.columns[2], 'Q2b': df.columns[2], \
                  'Q3a': df.columns[4], 'Q3b': df.columns[4], 'Q4': df.columns[6]}

In [13]:
long_col_names

{'Q1a': 'What are the TWO major challenges faced by embryologists in the workplace?',
 'Q1b': 'What are the TWO major challenges faced by embryologists in the workplace?',
 'Q2a': 'What are the TWO major challenges faced by the embryology profession, in general? ',
 'Q2b': 'What are the TWO major challenges faced by the embryology profession, in general? ',
 'Q3a': "Provide TWO suggestions to improve embryologists' working conditions",
 'Q3b': "Provide TWO suggestions to improve embryologists' working conditions",
 'Q4': 'What is your career goal?'}

In [15]:
# rename the columns
df = df.rename(columns={df.columns[0]: 'Q1a', df.columns[1]: 'Q1b', df.columns[2]: 'Q2a',df.columns[3]: \
                        'Q2b', df.columns[4]: 'Q3a', df.columns[5]: 'Q3b', df.columns[6]: 'Q4'})

In [17]:
# remove first row
df = df.iloc[1:]

In [18]:
df

,Q1a,Q1b,Q2a,Q2b,Q3a,Q3b,Q4
1,Equality with clinical members,Poor pay,Shortage of trained staff,Equality,Improve number,Improve pay,Continue
2,Working hours and too many responsibilities,Salary,Access to training,Certification,Add reading and projects into basic workload n...,Understand for how many hours a brain can be f...,Get my ESHRE certification Publish paper thro...
3,burnout/stress,poor management,not enough highly trained staff,politics,better pay,better CPD opportunities,FRCPath
4,Recognizion,Trust,Handson training,Trust,Good laboratory training,Troubleshooting,Be confident in the work i do Academically st...
5,General workplace politics,Internal Pressure,Undervalued,Underpaid,Flexible hours to reduce stress,Shorter rotations in the Lab,I would like to work as an embryologist intern...
...,...,...,...,...,...,...,...
853,Low Salaries,Working weekends,Not closing over December,Low Salaries,Higher salaries,Closing over December,To complete my masters and become the best pra...
854,low salaries,weekend and holiday work,low salaries,weekend and holiday work,increase salaries,close over christmas,Dont make a big mistake
855,Low salaries,Weekend and holiday work,Low salary,Weekend and holiday work,Higher salaries,Close over Christmas holidays,Make babies and make people happy.
856,Respect from other departments for the complex...,Financial compensation for overtime etc.,Financial participation: Owning shares,lack of respect for the work we do by doctors,Pay them!,Invite financial incentives to the embryologis...,I reached the glass ceiling. All I can do is h...


In [33]:
col='Q1a'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by embryologists in the workplace?


,Q1a,sentiment
1,Equality with clinical members,0.0000
2,Working hours and too many responsibilities,0.0000
3,burnout/stress,0.0000
4,Recognizion,0.0000
5,General workplace politics,0.0000
...,...,...
853,Low Salaries,-0.2732
854,low salaries,-0.2732
855,Low salaries,-0.2732
856,Respect from other departments for the complex work we do,0.4767


Average sentiment: -0.047
Std dev:            0.240


In [34]:
col='Q1b'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by embryologists in the workplace?


,Q1b,sentiment
1,Poor pay,-0.5423
2,Salary,0.0000
3,poor management,-0.4767
4,Trust,0.5106
5,Internal Pressure,-0.2960
...,...,...
853,Working weekends,0.0000
854,weekend and holiday work,0.4019
855,Weekend and holiday work,0.4019
856,Financial compensation for overtime etc.,0.0000


Average sentiment: -0.048
Std dev:            0.240


In [35]:
col='Q2a'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by the embryology profession, in general? 


,Q2a,sentiment
1,Shortage of trained staff,-0.2500
2,Access to training,0.0000
3,not enough highly trained staff,0.0000
4,Handson training,0.0000
5,Undervalued,0.0000
...,...,...
853,Not closing over December,0.0000
854,low salaries,-0.2732
855,Low salary,-0.2732
856,Financial participation: Owning shares,0.2960


Average sentiment: -0.031
Std dev:            0.226


In [36]:
col='Q2b'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by the embryology profession, in general? 


,Q2b,sentiment
1,Equality,0.0000
2,Certification,0.0000
3,politics,0.0000
4,Trust,0.5106
5,Underpaid,0.0000
...,...,...
853,Low Salaries,-0.2732
854,weekend and holiday work,0.4019
855,Weekend and holiday work,0.4019
856,lack of respect for the work we do by doctors,0.2023


Average sentiment: -0.042
Std dev:            0.241


In [37]:
col='Q3a'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:15.3f}".format(df.loc[:, 'sentiment'].std()))

Provide TWO suggestions to improve embryologists' working conditions


,Q3a,sentiment
1,Improve number,0.4939
2,Add reading and projects into basic workload not in leisure time,0.0000
3,better pay,0.3612
4,Good laboratory training,0.4404
5,Flexible hours to reduce stress,-0.2263
...,...,...
853,Higher salaries,0.0000
854,increase salaries,0.3182
855,Higher salaries,0.0000
856,Pay them!,-0.1759


Average sentiment: 0.125
Std dev:           0.223


In [38]:
col='Q3b'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:15.3f}".format(df.loc[:, 'sentiment'].std()))

Provide TWO suggestions to improve embryologists' working conditions


,Q3b,sentiment
1,Improve pay,0.3612
2,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
3,better CPD opportunities,0.6705
4,Troubleshooting,0.1779
5,Shorter rotations in the Lab,0.0000
...,...,...
853,Closing over December,0.0000
854,close over christmas,0.0000
855,Close over Christmas holidays,0.3818
856,Invite financial incentives to the embryologists as well. e.g. Shares!,0.7574


Average sentiment: 0.150
Std dev:           0.242


In [39]:
col='Q4'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:15.3f}".format(df.loc[:, 'sentiment'].std()))

What is your career goal?


,Q4,sentiment
1,Continue,0.0000
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
3,FRCPath,0.0000
4,Be confident in the work i do Academically strong,0.7579
5,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
...,...,...
853,To complete my masters and become the best practical embryologist to my abilities.,0.7351
854,Dont make a big mistake,0.2584
855,Make babies and make people happy.,0.5719
856,I reached the glass ceiling. All I can do is hope to study further and further and further.....,0.5106


Average sentiment: 0.190
Std dev:           0.306


In [41]:
# read csv ignoring potential N/As
df = pd.read_csv('Data/male.csv', na_filter = False)

In [42]:
df

,What are the TWO major challenges faced by embryologists in the workplace?,Unnamed: 1,"What are the TWO major challenges faced by the embryology profession, in general?",Unnamed: 3,Provide TWO suggestions to improve embryologists' working conditions,Unnamed: 5,What is your career goal?
0,1,2,1,2,1,2,Open-Ended Response
1,Bullying by colleagues and managers,"Poorly designed protocols, technical ignorance...",Stress of delivering high quality work without...,Presence of narcissistic individuals destroyin...,Screening for sociopathic personality traits n...,Clinics must be staffed so that there is a goo...,To survive until retirement without suffering ...
2,Stress created by no embryos/ less embryos/ po...,No respect,Gossips of others which disturbs other Embryol...,No respect,Reporting Manager should understand the situation,Stress free atmosphere,To be Good Embryologist by providing maximum ...
3,lack of leadership,burnout,managment,AI,train managers to lead and not manage,recognition,have a happy team
4,work overload,No/less support from clinicians,Poor salary,Expected to work hard and give good results,Constant support form clinicians,Increase salaries or pay better and relative t...,To achieve high position and personal developm...
...,...,...,...,...,...,...,...
381,Access to knowledge about QC / QA and the tool...,Learning embryo biopsy,Earn a salary according to the profession,Professional recognition,Improve access to training,Improve wages,Have the best reproductive results in my country
382,"Motivation (stress, burnout)",Getting the work done,Safety in the lab,Deliver quality,even distribution of work,provide sufficient staff,PhD
383,"not getting enough working materials, in ethiopia",to get Professional maintenance is difficult i...,maintaining the quality control in the IVF lab,"training of good embryology staffs,",having good embryology training center,support technicians financially to get trainin...,be ART researcher.
384,Too many cases per embryologist,Not enough off-time,Continuous skill development,Not enough staff,More staff to allow more personal time,More opportunities to better our knowledge,To be the best embryologist I can be to the pa...


In [43]:
# create a dict for the short and log column names
long_col_names = {'Q1a': df.columns[0], 'Q1b': df.columns[0], 'Q2a': df.columns[2], 'Q2b': df.columns[2], \
                  'Q3a': df.columns[4], 'Q3b': df.columns[4], 'Q4': df.columns[6]}

In [44]:
long_col_names

{'Q1a': 'What are the TWO major challenges faced by embryologists in the workplace?',
 'Q1b': 'What are the TWO major challenges faced by embryologists in the workplace?',
 'Q2a': 'What are the TWO major challenges faced by the embryology profession, in general? ',
 'Q2b': 'What are the TWO major challenges faced by the embryology profession, in general? ',
 'Q3a': "Provide TWO suggestions to improve embryologists' working conditions",
 'Q3b': "Provide TWO suggestions to improve embryologists' working conditions",
 'Q4': 'What is your career goal?'}

In [45]:
# rename the columns
df = df.rename(columns={df.columns[0]: 'Q1a', df.columns[1]: 'Q1b', df.columns[2]: 'Q2a',df.columns[3]: \
                        'Q2b', df.columns[4]: 'Q3a', df.columns[5]: 'Q3b', df.columns[6]: 'Q4'})

In [46]:
# remove first row
df = df.iloc[1:]

In [47]:
df

,Q1a,Q1b,Q2a,Q2b,Q3a,Q3b,Q4
1,Bullying by colleagues and managers,"Poorly designed protocols, technical ignorance...",Stress of delivering high quality work without...,Presence of narcissistic individuals destroyin...,Screening for sociopathic personality traits n...,Clinics must be staffed so that there is a goo...,To survive until retirement without suffering ...
2,Stress created by no embryos/ less embryos/ po...,No respect,Gossips of others which disturbs other Embryol...,No respect,Reporting Manager should understand the situation,Stress free atmosphere,To be Good Embryologist by providing maximum ...
3,lack of leadership,burnout,managment,AI,train managers to lead and not manage,recognition,have a happy team
4,work overload,No/less support from clinicians,Poor salary,Expected to work hard and give good results,Constant support form clinicians,Increase salaries or pay better and relative t...,To achieve high position and personal developm...
5,Time spent at work,Money for new quipment,Low salary,Stress,More staff,more off time,Best success rate to promote our clinic
...,...,...,...,...,...,...,...
381,Access to knowledge about QC / QA and the tool...,Learning embryo biopsy,Earn a salary according to the profession,Professional recognition,Improve access to training,Improve wages,Have the best reproductive results in my country
382,"Motivation (stress, burnout)",Getting the work done,Safety in the lab,Deliver quality,even distribution of work,provide sufficient staff,PhD
383,"not getting enough working materials, in ethiopia",to get Professional maintenance is difficult i...,maintaining the quality control in the IVF lab,"training of good embryology staffs,",having good embryology training center,support technicians financially to get trainin...,be ART researcher.
384,Too many cases per embryologist,Not enough off-time,Continuous skill development,Not enough staff,More staff to allow more personal time,More opportunities to better our knowledge,To be the best embryologist I can be to the pa...


In [48]:
col='Q1a'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by embryologists in the workplace?


,Q1a,sentiment
1,Bullying by colleagues and managers,-0.5994
2,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024
3,lack of leadership,-0.3182
4,work overload,-0.3612
5,Time spent at work,0.0000
...,...,...
381,Access to knowledge about QC / QA and the tools to apply them,0.0000
382,"Motivation (stress, burnout)",0.3400
383,"not getting enough working materials, in ethiopia",0.0000
384,Too many cases per embryologist,0.0000


Average sentiment: -0.037
Std dev:            0.237


In [49]:
col='Q1b'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by embryologists in the workplace?


,Q1b,sentiment
1,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
2,No respect,0.2263
3,burnout,0.0000
4,No/less support from clinicians,0.4019
5,Money for new quipment,0.0000
...,...,...
381,Learning embryo biopsy,0.0000
382,Getting the work done,0.0000
383,to get Professional maintenance is difficult in my area,-0.3612
384,Not enough off-time,0.0000


Average sentiment: -0.054
Std dev:            0.251


In [50]:
col='Q2a'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by the embryology profession, in general? 


,Q2a,sentiment
1,Stress of delivering high quality work without sufficient staff.,-0.4215
2,Gossips of others which disturbs other Embryologist work,-0.6369
3,managment,0.0000
4,Poor salary,-0.4767
5,Low salary,-0.2732
...,...,...
381,Earn a salary according to the profession,0.0000
382,Safety in the lab,0.4215
383,maintaining the quality control in the IVF lab,0.0000
384,Continuous skill development,0.0000


Average sentiment: -0.040
Std dev:            0.241


In [51]:
col='Q2b'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:16.3f}".format(df.loc[:, 'sentiment'].std()))

What are the TWO major challenges faced by the embryology profession, in general? 


,Q2b,sentiment
1,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
2,No respect,0.2263
3,AI,0.0000
4,Expected to work hard and give good results,0.3612
5,Stress,-0.4215
...,...,...
381,Professional recognition,0.0000
382,Deliver quality,0.0000
383,"training of good embryology staffs,",0.4404
384,Not enough staff,0.0000


Average sentiment: -0.024
Std dev:            0.231


In [52]:
col='Q3a'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:15.3f}".format(df.loc[:, 'sentiment'].std()))

Provide TWO suggestions to improve embryologists' working conditions


,Q3a,sentiment
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960
2,Reporting Manager should understand the situation,0.0000
3,train managers to lead and not manage,0.0000
4,Constant support form clinicians,0.4019
5,More staff,0.0000
...,...,...
381,Improve access to training,0.4404
382,even distribution of work,0.0000
383,having good embryology training center,0.4404
384,More staff to allow more personal time,0.2878


Average sentiment: 0.128
Std dev:           0.222


In [53]:
col='Q3b'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:15.3f}".format(df.loc[:, 'sentiment'].std()))

Provide TWO suggestions to improve embryologists' working conditions


,Q3b,sentiment
1,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
2,Stress free atmosphere,0.1280
3,recognition,0.0000
4,Increase salaries or pay better and relative to the workload,0.5859
5,more off time,0.0000
...,...,...
381,Improve wages,0.4404
382,provide sufficient staff,0.0000
383,support technicians financially to get training abroad,0.4019
384,More opportunities to better our knowledge,0.7233


Average sentiment: 0.145
Std dev:           0.236


In [54]:
col='Q4'
print(long_col_names[col])
df['sentiment'] = df[col].apply(get_compound_sentiment)
with pd.option_context('display.max_colwidth', None):
    display(df[[col, 'sentiment']])
print("Average sentiment: {0:2.3f}".format(df.loc[:, 'sentiment'].mean()))
print("Std dev: {0:15.3f}".format(df.loc[:, 'sentiment'].std()))

What is your career goal?


,Q4,sentiment
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
2,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
3,have a happy team,0.5719
4,To achieve high position and personal development (lab director or chief technologist),0.0000
5,Best success rate to promote our clinic,0.8885
...,...,...
381,Have the best reproductive results in my country,0.6369
382,PhD,0.0000
383,be ART researcher.,0.0000
384,To be the best embryologist I can be to the patients.,0.6369


Average sentiment: 0.213
Std dev:           0.307


| Q                 | Female   | Male| Average |
| :---------------- | :------: | ----: | ----: |
| Q1a               |   -0.047  | -0.037 | -0.045 |
| Q1b               |   -0.048  | -0.054 | -0.051 |
| Q2a               |   -0.031  | -0.040 | -0.035 |
| Q2b               |   -0.042  | -0.024 | -0.038 |
| Q3a               |   0.125   | 0.128 | 0.126 |
| Q3b               |   0.150   | 0.145 | 0.148 |
| Q4               |   0.190   | 0.213 | 0.198 |

## 3. Gender

In [369]:
gender_results = {'Gender': ['Male', 'Female'],
        'Q1': [0.1, 0.2],
        'Q2': [0.1, 0.2],
        'Q3': [0.1, 0.2],
        'Q4': [0.1, 0.2]}
gender_results_df = pd.DataFrame.from_dict(gen_results)

In [370]:
gender_results_df

,Gender,Q1,Q2,Q3,Q4
0,Male,0.1,0.1,0.1,0.1
1,Female,0.2,0.2,0.2,0.2


### 3.1 Male

In [344]:
male_df = df[df["Gender"].isin(["Male"])]
analyse_verbatims(0, male_df, gender_results_df)
gender_results_df

,Q1a,sentiment1,Q1b,sentiment2
1,Bullying by colleagues and managers,-0.5994,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
5,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024,No respect,0.2263
9,lack of leadership,-0.3182,burnout,0.0000
13,work overload,-0.3612,No/less support from clinicians,0.4019
14,Time spent at work,0.0000,Money for new quipment,0.0000
...,...,...,...,...
1241,Access to knowledge about QC / QA and the tools to apply them,0.0000,Learning embryo biopsy,0.0000
1244,"Motivation (stress, burnout)",0.3400,Getting the work done,0.0000
1246,"not getting enough working materials, in ethiopia",0.0000,to get Professional maintenance is difficult in my area,-0.3612
1247,Too many cases per embryologist,0.0000,Not enough off-time,0.0000


Mean sentiment: -0.045
Std dev:         0.244


,Q2a,sentiment1,Q2b,sentiment2
1,Stress of delivering high quality work without sufficient staff.,-0.4215,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
5,Gossips of others which disturbs other Embryologist work,-0.6369,No respect,0.2263
9,managment,0.0000,AI,0.0000
13,Poor salary,-0.4767,Expected to work hard and give good results,0.3612
14,Low salary,-0.2732,Stress,-0.4215
...,...,...,...,...
1241,Earn a salary according to the profession,0.0000,Professional recognition,0.0000
1244,Safety in the lab,0.4215,Deliver quality,0.0000
1246,maintaining the quality control in the IVF lab,0.0000,"training of good embryology staffs,",0.4404
1247,Continuous skill development,0.0000,Not enough staff,0.0000


Mean sentiment: -0.032
Std dev:         0.236


,Q3a,sentiment1,Q3b,sentiment2
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
5,Reporting Manager should understand the situation,0.0000,Stress free atmosphere,0.1280
9,train managers to lead and not manage,0.0000,recognition,0.0000
13,Constant support form clinicians,0.4019,Increase salaries or pay better and relative to the workload,0.5859
14,More staff,0.0000,more off time,0.0000
...,...,...,...,...
1241,Improve access to training,0.4404,Improve wages,0.4404
1244,even distribution of work,0.0000,provide sufficient staff,0.0000
1246,having good embryology training center,0.4404,support technicians financially to get training abroad,0.4019
1247,More staff to allow more personal time,0.2878,More opportunities to better our knowledge,0.7233


Mean sentiment:  0.136
Std dev:         0.229


,Q4,sentiment
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
5,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
9,have a happy team,0.5719
13,To achieve high position and personal development (lab director or chief technologist),0.0000
14,Best success rate to promote our clinic,0.8885
...,...,...
1241,Have the best reproductive results in my country,0.6369
1244,PhD,0.0000
1246,be ART researcher.,0.0000
1247,To be the best embryologist I can be to the patients.,0.6369


Mean sentiment:  0.213
Std dev:         0.307


,Gender,Q1,Q2,Q3,Q4
0,Male,-0.045247,-0.03162,0.136469,0.212677
1,Female,-0.047248,-0.03609,0.137173,0.189534


### 3.1 Female

In [345]:
female_df = df[df["Gender"].isin(["Female"])]
analyse_verbatims(1, female_df, gender_results_df)
gender_results_df

,Q1a,sentiment1,Q1b,sentiment2
0,Equality with clinical members,0.0000,Poor pay,-0.5423
2,Working hours and too many responsibilities,0.0000,Salary,0.0000
3,burnout/stress,0.0000,poor management,-0.4767
4,Recognizion,0.0000,Trust,0.5106
6,General workplace politics,0.0000,Internal Pressure,-0.2960
...,...,...,...,...
1250,Low Salaries,-0.2732,Working weekends,0.0000
1251,low salaries,-0.2732,weekend and holiday work,0.4019
1252,Low salaries,-0.2732,Weekend and holiday work,0.4019
1253,Respect from other departments for the complex work we do,0.4767,Financial compensation for overtime etc.,0.0000


Mean sentiment: -0.047
Std dev:         0.240


,Q2a,sentiment1,Q2b,sentiment2
0,Shortage of trained staff,-0.2500,Equality,0.0000
2,Access to training,0.0000,Certification,0.0000
3,not enough highly trained staff,0.0000,politics,0.0000
4,Handson training,0.0000,Trust,0.5106
6,Undervalued,0.0000,Underpaid,0.0000
...,...,...,...,...
1250,Not closing over December,0.0000,Low Salaries,-0.2732
1251,low salaries,-0.2732,weekend and holiday work,0.4019
1252,Low salary,-0.2732,Weekend and holiday work,0.4019
1253,Financial participation: Owning shares,0.2960,lack of respect for the work we do by doctors,0.2023


Mean sentiment: -0.036
Std dev:         0.234


,Q3a,sentiment1,Q3b,sentiment2
0,Improve number,0.4939,Improve pay,0.3612
2,Add reading and projects into basic workload not in leisure time,0.0000,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
3,better pay,0.3612,better CPD opportunities,0.6705
4,Good laboratory training,0.4404,Troubleshooting,0.1779
6,Flexible hours to reduce stress,-0.2263,Shorter rotations in the Lab,0.0000
...,...,...,...,...
1250,Higher salaries,0.0000,Closing over December,0.0000
1251,increase salaries,0.3182,close over christmas,0.0000
1252,Higher salaries,0.0000,Close over Christmas holidays,0.3818
1253,Pay them!,-0.1759,Invite financial incentives to the embryologists as well. e.g. Shares!,0.7574


Mean sentiment:  0.137
Std dev:         0.232


,Q4,sentiment
0,Continue,0.0000
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
3,FRCPath,0.0000
4,Be confident in the work i do Academically strong,0.7579
6,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
...,...,...
1250,To complete my masters and become the best practical embryologist to my abilities.,0.7351
1251,Dont make a big mistake,0.2584
1252,Make babies and make people happy.,0.5719
1253,I reached the glass ceiling. All I can do is hope to study further and further and further.....,0.5106


Mean sentiment:  0.190
Std dev:         0.306


,Gender,Q1,Q2,Q3,Q4
0,Male,-0.045247,-0.03162,0.136469,0.212677
1,Female,-0.047248,-0.03609,0.137173,0.189534


### Show heatmap

In [364]:
import seaborn as sns
# cm = sns.light_palette("green", as_cmap=True)
# s = cont_results_df.style.background_gradient(cmap='viridis')
s = gender_results_df.style.background_gradient()
s

,Gender,Q1,Q2,Q3,Q4
0,Male,-0.045247,-0.031620,0.136469,0.212677
1,Female,-0.047248,-0.036090,0.137173,0.189534


## 4. World bank classification

In [407]:
WBclass_results = {'World bank classification': ['High-income economies', 'Upper-middle-income economies', \
                                                 'Lower-middle-income economies', 'Low-income economies' ],
        'Q1': [0.1, 0.2, 0.3, 0.4],
        'Q2': [0.1, 0.2, 0.3, 0.4],
        'Q3': [0.1, 0.2, 0.3, 0.4],
        'Q4': [0.1, 0.2, 0.3, 0.4]}
WBclass_results_df = pd.DataFrame.from_dict(WBclass_results)
WBclass_results_df

,World bank classification,Q1,Q2,Q3,Q4
0,High-income economies,0.1,0.1,0.1,0.1
1,Upper-middle-income economies,0.2,0.2,0.2,0.2
2,Lower-middle-income economies,0.3,0.3,0.3,0.3
3,Low-income economies,0.4,0.4,0.4,0.4


In [397]:
hie_df = df[df["Category 2: World bank classification"].isin(['High-income economies '])]
analyse_verbatims(0, hie_df, WBclass_results_df)
WBclass_results_df

,Q1a,sentiment1,Q1b,sentiment2
0,Equality with clinical members,0.0000,Poor pay,-0.5423
1,Bullying by colleagues and managers,-0.5994,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
2,Working hours and too many responsibilities,0.0000,Salary,0.0000
3,burnout/stress,0.0000,poor management,-0.4767
9,lack of leadership,-0.3182,burnout,0.0000
...,...,...,...,...
1237,oocyte vitrification,0.0000,more sophisticated witnessing - software etc.,0.5984
1238,Quality control,0.0000,Work pressure,-0.2960
1240,Lack of consideration by MDs,-0.3182,Understaffing,0.0000
1244,"Motivation (stress, burnout)",0.3400,Getting the work done,0.0000


Mean sentiment: -0.059
Std dev:         0.230


,Q2a,sentiment1,Q2b,sentiment2
0,Shortage of trained staff,-0.2500,Equality,0.0000
1,Stress of delivering high quality work without sufficient staff.,-0.4215,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
2,Access to training,0.0000,Certification,0.0000
3,not enough highly trained staff,0.0000,politics,0.0000
9,managment,0.0000,AI,0.0000
...,...,...,...,...
1237,AI for embryo - selection,0.0000,NiPGT,0.0000
1238,Quality management control,0.0000,Quality management assurance,0.3400
1240,Undervalued,0.0000,Underpaid,0.0000
1244,Safety in the lab,0.4215,Deliver quality,0.0000


Mean sentiment: -0.032
Std dev:         0.221


,Q3a,sentiment1,Q3b,sentiment2
0,Improve number,0.4939,Improve pay,0.3612
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
2,Add reading and projects into basic workload not in leisure time,0.0000,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
3,better pay,0.3612,better CPD opportunities,0.6705
9,train managers to lead and not manage,0.0000,recognition,0.0000
...,...,...,...,...
1237,IVF lab facility,0.0000,good collective,0.4404
1238,Administrative simplification,0.0000,Clinic-laboratory communication,0.0000
1240,Staffing guidelines/rules,0.0000,Educational guidelines and rules,0.0000
1244,even distribution of work,0.0000,provide sufficient staff,0.0000


Mean sentiment:  0.128
Std dev:         0.223


,Q4,sentiment
0,Continue,0.0000
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
3,FRCPath,0.0000
9,have a happy team,0.5719
...,...,...
1237,I am satisfied :-),0.6249
1238,Senior clinical embryologist,0.0000
1240,Retire with money in the bank,0.0000
1244,PhD,0.0000


Mean sentiment:  0.150
Std dev:         0.289


,World bank classification,Q1,Q2,Q3,Q4
0,High-income economies,-0.058991,-0.031771,0.128162,0.149908
1,Upper-middle-income economies,0.200000,0.200000,0.200000,0.200000
2,Lower-middle-income economies,0.300000,0.300000,0.300000,0.300000
3,Low-income economies,0.400000,0.400000,0.400000,0.400000


In [398]:
umie_df = df[df["Category 2: World bank classification"].isin(['Upper-middle-income economies'])]
analyse_verbatims(1, umie_df, WBclass_results_df)
WBclass_results_df

,Q1a,sentiment1,Q1b,sentiment2
6,General workplace politics,0.0000,Internal Pressure,-0.2960
7,Burnout due to working continously through weekends without breaks,0.0000,Difficulty in taking time off due to understaffed clinics,-0.3400
8,fertilization outcomes are laid on the embryologists,0.0000,pregnancies outcomes are laid on the embryologists,0.0000
10,Recognition of our contribution in the ART field,0.0000,Workload,0.0000
11,Time managment,0.0000,Long hours,0.0000
...,...,...,...,...
1250,Low Salaries,-0.2732,Working weekends,0.0000
1251,low salaries,-0.2732,weekend and holiday work,0.4019
1252,Low salaries,-0.2732,Weekend and holiday work,0.4019
1253,Respect from other departments for the complex work we do,0.4767,Financial compensation for overtime etc.,0.0000


Mean sentiment: -0.025
Std dev:         0.230


,Q2a,sentiment1,Q2b,sentiment2
6,Undervalued,0.0000,Underpaid,0.0000
7,Lack of qualified embryologists - staff shortages,-0.4404,Underpayment for specialized skills,0.0000
8,overworking with minimum breaks (leave),0.0000,overall outcome of the patient pregnancy rates are laid on the embryologists,0.0000
10,Inferior treatment by Clinicians,-0.4019,Workload,0.0000
11,Lack of respect,0.2023,Having to make do with bare min equipment,0.0000
...,...,...,...,...
1250,Not closing over December,0.0000,Low Salaries,-0.2732
1251,low salaries,-0.2732,weekend and holiday work,0.4019
1252,Low salary,-0.2732,Weekend and holiday work,0.4019
1253,Financial participation: Owning shares,0.2960,lack of respect for the work we do by doctors,0.2023


Mean sentiment: -0.048
Std dev:         0.236


,Q3a,sentiment1,Q3b,sentiment2
6,Flexible hours to reduce stress,-0.2263,Shorter rotations in the Lab,0.0000
7,"Allow for more clinics to be accredited training units, thus increasing accessibility to science graduates and allowing more embryologists into the field",0.2263,"If an embryologist has worked on a weekend, allowing one day off during the week",0.0000
8,improve trust and provide fair team work to embryologist from the clinicians,0.8176,allow reasonable breaks to the embryologist for resting (leave),0.2263
10,Awareness by Clinic owners,0.0000,Educate us how to successfully get what we are asking for,0.4939
11,Bigger budget for new equipment,0.0000,avoidance of working both days on weekends,-0.4019
...,...,...,...,...
1250,Higher salaries,0.0000,Closing over December,0.0000
1251,increase salaries,0.3182,close over christmas,0.0000
1252,Higher salaries,0.0000,Close over Christmas holidays,0.3818
1253,Pay them!,-0.1759,Invite financial incentives to the embryologists as well. e.g. Shares!,0.7574


Mean sentiment:  0.144
Std dev:         0.236


,Q4,sentiment
6,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
7,To be stable and happy while doing a job that I love.,0.8779
8,"to run an independant practice, be a partner and not an employee",0.0000
10,This is more of an aspiration - I want my unit to be the best in the country,0.6705
11,Done it,0.0000
...,...,...
1250,To complete my masters and become the best practical embryologist to my abilities.,0.7351
1251,Dont make a big mistake,0.2584
1252,Make babies and make people happy.,0.5719
1253,I reached the glass ceiling. All I can do is hope to study further and further and further.....,0.5106


Mean sentiment:  0.219
Std dev:         0.313


,World bank classification,Q1,Q2,Q3,Q4
0,High-income economies,-0.058991,-0.031771,0.128162,0.149908
1,Upper-middle-income economies,-0.025155,-0.048004,0.143765,0.218776
2,Lower-middle-income economies,0.300000,0.300000,0.300000,0.300000
3,Low-income economies,0.400000,0.400000,0.400000,0.400000


In [399]:
lmie_df = df[df["Category 2: World bank classification"].isin(['Lower-middle-income economies '])]
analyse_verbatims(2, lmie_df, WBclass_results_df)
WBclass_results_df

,Q1a,sentiment1,Q1b,sentiment2
4,Recognizion,0.0000,Trust,0.5106
5,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024,No respect,0.2263
25,Employees shortage,-0.2500,Untrained staff,0.0000
39,Little black of communication with clinical team,0.0000,Working hours,0.0000
44,Stress,-0.4215,trouble shoot,-0.6249
...,...,...,...,...
1158,Results,0.0000,Waiting time,0.0000
1159,Day-5 blastocyst formation,0.0000,Thawed semen sample preparation,0.0000
1160,Interference of the clinician,0.0000,No credits for positives,0.5719
1161,Number of cases,0.0772,Quality Management,0.0000


Mean sentiment: -0.042
Std dev:         0.274


,Q2a,sentiment1,Q2b,sentiment2
4,Handson training,0.0000,Trust,0.5106
5,Gossips of others which disturbs other Embryologist work,-0.6369,No respect,0.2263
25,Oocyte quality,0.0000,Implantation even after everything is good,0.4404
39,Time at workplace,0.0000,Little stress for achieving better outcome,0.1010
44,stress,-0.4215,results,0.0000
...,...,...,...,...
1158,NA,0.0000,NA,0.0000
1159,Thawed embryos results,0.0000,D-5 blastocyst formation or grading,0.0000
1160,Full time responsibility without any holidays,-0.2924,Paid less,0.0000
1161,Number of cases,0.0772,Quality Management,0.0000


Mean sentiment: -0.039
Std dev:         0.262


,Q3a,sentiment1,Q3b,sentiment2
4,Good laboratory training,0.4404,Troubleshooting,0.1779
5,Reporting Manager should understand the situation,0.0000,Stress free atmosphere,0.1280
25,Should be adequate staff,0.2263,Time flexibility,0.3400
39,Good communication with team,0.4404,Fixed working hours and open discussion,0.0000
44,diluting the work load,0.0000,Stress relief,0.0772
...,...,...,...,...
1158,NA,0.0000,NA,0.0000
1159,Avoid exposure of embryos as much you can,-0.2960,Assess embryos on day0- day1 and then on day 3 or 5 only,0.0000
1160,Definite working hours,0.2732,Proper trainings,0.0000
1161,Provide optimum number of embryologist depending on the number of cases,0.1531,Software to ease the data maintenance,0.3612


Mean sentiment:  0.151
Std dev:         0.244


,Q4,sentiment
4,Be confident in the work i do Academically strong,0.7579
5,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
25,To find answers of failed implantation,-0.5106
39,To give my best,0.6369
44,"Academic excellence, lead role, research",0.6249
...,...,...
1158,Embryologist,0.0000
1159,Become a successful scientist,0.5859
1160,To help as many infertile couple as possible & to give them a healthy baby of their own.,0.6597
1161,To become a successful embryologist,0.5859


Mean sentiment:  0.285
Std dev:         0.318


,World bank classification,Q1,Q2,Q3,Q4
0,High-income economies,-0.058991,-0.031771,0.128162,0.149908
1,Upper-middle-income economies,-0.025155,-0.048004,0.143765,0.218776
2,Lower-middle-income economies,-0.042376,-0.038700,0.151379,0.284706
3,Low-income economies,0.400000,0.400000,0.400000,0.400000


In [401]:
lie_df = df[df["Category 2: World bank classification"].isin(['Low-income economies '])]
analyse_verbatims(3, lie_df, WBclass_results_df)
WBclass_results_df

,Q1a,sentiment1,Q1b,sentiment2
428,deal with everyday work,0.0000,relations with doctors,0.0000
499,low pay,-0.3612,un appreciated by clinicians,0.5106
635,Scientific decision making against Clinical stands,0.0000,Clinical influences on the gamates (oocytes) quality,0.0000
989,Salary satisfaction,0.4404,Career development,0.0000
1058,Low payment,-0.2732,Low respect for Juniors,0.2500
1246,"not getting enough working materials, in ethiopia",0.0000,to get Professional maintenance is difficult in my area,-0.3612


Mean sentiment:  0.017
Std dev:         0.286


,Q2a,sentiment1,Q2b,sentiment2
428,doctors,0.0000,how the government recognises us,0.0000
499,long work hours,0.0000,lack of opportunities for reskilling,0.0772
635,Burden of proof that results are not your making,-0.4404,Decisions on embryo transfers policies.,0.0000
989,Career development,0.0000,Research support,0.4019
1058,No Regulations of work,-0.2960,Limited skills development centers in the country,-0.2263
1246,maintaining the quality control in the IVF lab,0.0000,"training of good embryology staffs,",0.4404


Mean sentiment: -0.004
Std dev:         0.227


,Q3a,sentiment1,Q3b,sentiment2
428,to make us worth it by not accepting all the conditions,-0.0731,to fight for our wills,-0.3818
499,Networking and forming embryology societies to guide practice.,0.0000,training and reskilling,0.0000
635,Professional bodies to protect the embryologists decision,0.3818,Allow the embryologist to determine how much work they can carry.,0.2263
989,Continuous learning,0.0000,Motivation for development,0.3400
1058,Need for a regulating body,0.0000,Setting up more training centers in the country,0.0000
1246,having good embryology training center,0.4404,support technicians financially to get training abroad,0.4019


Mean sentiment:  0.111
Std dev:         0.257


,Q4,sentiment
428,to survive,0.0000
499,To practice evidence based embryology to the satisfaction of clients.,0.4404
635,Retire at 55years and go into academic/scientific research.,0.0000
989,Excellence,0.6249
1058,Becoming one of the best embryologist in my country and train more embryologist.,0.6369
1246,be ART researcher.,0.0000


Mean sentiment:  0.284
Std dev:         0.318


,World bank classification,Q1,Q2,Q3,Q4
0,High-income economies,-0.058991,-0.031771,0.128162,0.149908
1,Upper-middle-income economies,-0.025155,-0.048004,0.143765,0.218776
2,Lower-middle-income economies,-0.042376,-0.038700,0.151379,0.284706
3,Low-income economies,0.017117,-0.003600,0.111292,0.283700


### Show heatmap

In [402]:
import seaborn as sns
# cm = sns.light_palette("green", as_cmap=True)
# s = cont_results_df.style.background_gradient(cmap='viridis')
s = WBclass_results_df.style.background_gradient()
s

,World bank classification,Q1,Q2,Q3,Q4
0,High-income economies,-0.058991,-0.031771,0.128162,0.149908
1,Upper-middle-income economies,-0.025155,-0.048004,0.143765,0.218776
2,Lower-middle-income economies,-0.042376,-0.038700,0.151379,0.284706
3,Low-income economies,0.017117,-0.003600,0.111292,0.283700


## 5. Type of unit

In [406]:
ToU_results = {'Type of Unit': ['Academic Public Hospitals', 'Private Unit', 'Unit of a Chain'],
        'Q1': [0.1, 0.2, 0.3],
        'Q2': [0.1, 0.2, 0.3],
        'Q3': [0.1, 0.2, 0.3],
        'Q4': [0.1, 0.2, 0.3]}
ToU_results_df = pd.DataFrame.from_dict(ToU_results)
ToU_results_df

,Type of Unit,Q1,Q2,Q3,Q4
0,Academic Public Hospitals,0.1,0.1,0.1,0.1
1,Private Unit,0.2,0.2,0.2,0.2
2,Unit of a Chain,0.3,0.3,0.3,0.3


In [412]:
aph_df = df[df["Category: Place of work"].isin(['Academic public hospital'])]
analyse_verbatims(0, aph_df, ToU_results_df)
ToU_results_df

,Q1a,sentiment1,Q1b,sentiment2
3,burnout/stress,0.0000,poor management,-0.4767
5,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024,No respect,0.2263
8,fertilization outcomes are laid on the embryologists,0.0000,pregnancies outcomes are laid on the embryologists,0.0000
9,lack of leadership,-0.3182,burnout,0.0000
15,Workload,0.0000,Underpaid,0.0000
...,...,...,...,...
1240,Lack of consideration by MDs,-0.3182,Understaffing,0.0000
1241,Access to knowledge about QC / QA and the tools to apply them,0.0000,Learning embryo biopsy,0.0000
1242,Increased workload,0.2732,Unpaid overtime and weekend work,0.0000
1245,Burnout,0.0000,Workload,0.0000


Mean sentiment: -0.063
Std dev:         0.229


,Q2a,sentiment1,Q2b,sentiment2
3,not enough highly trained staff,0.0000,politics,0.0000
5,Gossips of others which disturbs other Embryologist work,-0.6369,No respect,0.2263
8,overworking with minimum breaks (leave),0.0000,overall outcome of the patient pregnancy rates are laid on the embryologists,0.0000
9,managment,0.0000,AI,0.0000
15,Underpayment,0.0000,Lack of recognition,-0.3182
...,...,...,...,...
1240,Undervalued,0.0000,Underpaid,0.0000
1241,Earn a salary according to the profession,0.0000,Professional recognition,0.0000
1242,Major responsibility,0.0000,Not always heard,0.0000
1245,Patient/Society Lack of Knowledge,-0.3182,Lack of recognition,-0.3182


Mean sentiment: -0.058
Std dev:         0.240


,Q3a,sentiment1,Q3b,sentiment2
3,better pay,0.3612,better CPD opportunities,0.6705
5,Reporting Manager should understand the situation,0.0000,Stress free atmosphere,0.1280
8,improve trust and provide fair team work to embryologist from the clinicians,0.8176,allow reasonable breaks to the embryologist for resting (leave),0.2263
9,train managers to lead and not manage,0.0000,recognition,0.0000
15,Increase number of embryologists per lab,0.3818,Pay,-0.1027
...,...,...,...,...
1240,Staffing guidelines/rules,0.0000,Educational guidelines and rules,0.0000
1241,Improve access to training,0.4404,Improve wages,0.4404
1242,Overtime to a minimum,0.0000,Mindfullness of weekend work,0.0000
1245,"Guidelines for employers (hours, salary scales, overtime, benefits)",0.0000,Projects/Launches to improve Societal Knowledge about our profession,0.4404


Mean sentiment:  0.143
Std dev:         0.230


,Q4,sentiment
3,FRCPath,0.0000
5,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
8,"to run an independant practice, be a partner and not an employee",0.0000
9,have a happy team,0.5719
15,Help struggling couples to have family of their own.,-0.0258
...,...,...
1240,Retire with money in the bank,0.0000
1241,Have the best reproductive results in my country,0.6369
1242,I aspire to be a good technical and pratically skilled Embryologist with thorough knowledge and good patient communication and care skills.,0.8402
1245,Becoming the best embryologist I can be by refining my skills and learning more techniques or other skills. I would also like to complete my pHD,0.7717


Mean sentiment:  0.208
Std dev:         0.335


,Type of Unit,Q1,Q2,Q3,Q4
0,Academic Public Hospitals,-0.063179,-0.058399,0.143482,0.208301
1,Private Unit,0.200000,0.200000,0.200000,0.200000
2,Unit of a Chain,0.300000,0.300000,0.300000,0.300000


In [413]:
pu_df = df[df["Category: Place of work"].isin(['Private Unit'])]
analyse_verbatims(1, pu_df, ToU_results_df)
ToU_results_df

,Q1a,sentiment1,Q1b,sentiment2
1,Bullying by colleagues and managers,-0.5994,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
4,Recognizion,0.0000,Trust,0.5106
6,General workplace politics,0.0000,Internal Pressure,-0.2960
7,Burnout due to working continously through weekends without breaks,0.0000,Difficulty in taking time off due to understaffed clinics,-0.3400
10,Recognition of our contribution in the ART field,0.0000,Workload,0.0000
...,...,...,...,...
1253,Respect from other departments for the complex work we do,0.4767,Financial compensation for overtime etc.,0.0000
1254,limited staff,-0.2263,doctors,0.0000
1255,Training,0.0000,Being able to have technical and manual capacities,0.0000
1256,,0.0000,,0.0000


Mean sentiment: -0.041
Std dev:         0.243


,Q2a,sentiment1,Q2b,sentiment2
1,Stress of delivering high quality work without sufficient staff.,-0.4215,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
4,Handson training,0.0000,Trust,0.5106
6,Undervalued,0.0000,Underpaid,0.0000
7,Lack of qualified embryologists - staff shortages,-0.4404,Underpayment for specialized skills,0.0000
10,Inferior treatment by Clinicians,-0.4019,Workload,0.0000
...,...,...,...,...
1253,Financial participation: Owning shares,0.2960,lack of respect for the work we do by doctors,0.2023
1254,Salaries,0.0000,Limit work opportunities,0.3818
1255,There are no university programs for embryologiests,-0.2960,costs to be trained abroad,0.0000
1256,,0.0000,,0.0000


Mean sentiment: -0.033
Std dev:         0.235


,Q3a,sentiment1,Q3b,sentiment2
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
4,Good laboratory training,0.4404,Troubleshooting,0.1779
6,Flexible hours to reduce stress,-0.2263,Shorter rotations in the Lab,0.0000
7,"Allow for more clinics to be accredited training units, thus increasing accessibility to science graduates and allowing more embryologists into the field",0.2263,"If an embryologist has worked on a weekend, allowing one day off during the week",0.0000
10,Awareness by Clinic owners,0.0000,Educate us how to successfully get what we are asking for,0.4939
...,...,...,...,...
1253,Pay them!,-0.1759,Invite financial incentives to the embryologists as well. e.g. Shares!,0.7574
1254,Better salaries,0.4404,Flexible working hours,0.2263
1255,havng university programs in embryology,0.0000,having fellowships or grants to perform this programs,0.2263
1256,,0.0000,,0.0000


Mean sentiment:  0.133
Std dev:         0.231


,Q4,sentiment
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
4,Be confident in the work i do Academically strong,0.7579
6,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
7,To be stable and happy while doing a job that I love.,0.8779
10,This is more of an aspiration - I want my unit to be the best in the country,0.6705
...,...,...
1253,I reached the glass ceiling. All I can do is hope to study further and further and further.....,0.5106
1254,Laboratory director,0.0000
1255,being able to help our patients,0.4019
1256,,0.0000


Mean sentiment:  0.194
Std dev:         0.300


,Type of Unit,Q1,Q2,Q3,Q4
0,Academic Public Hospitals,-0.063179,-0.058399,0.143482,0.208301
1,Private Unit,-0.040890,-0.032674,0.133166,0.194092
2,Unit of a Chain,0.300000,0.300000,0.300000,0.300000


In [414]:
uoc_df = df[df["Category: Place of work"].isin(['Unit of a Chain'])]
analyse_verbatims(2, uoc_df, ToU_results_df)
ToU_results_df

,Q1a,sentiment1,Q1b,sentiment2
0,Equality with clinical members,0.0000,Poor pay,-0.5423
2,Working hours and too many responsibilities,0.0000,Salary,0.0000
28,make no mistake,-0.5574,manage time properly,0.0000
35,emotional stress,-0.2960,patients/doctors demands,0.0000
37,Heavy workload,0.0000,Stres,0.0000
...,...,...,...,...
1202,Under staffing,0.0000,Burnout,0.0000
1203,Not enough employees,0.0000,Time off work,0.0000
1218,Burn out,0.0000,Stress,-0.4215
1222,Scheduling,0.0000,Short staffed,0.0000


Mean sentiment: -0.058
Std dev:         0.248


,Q2a,sentiment1,Q2b,sentiment2
0,Shortage of trained staff,-0.2500,Equality,0.0000
2,Access to training,0.0000,Certification,0.0000
28,professional title evaluation,0.0000,paper publishing,0.0000
35,emotional stress,-0.2960,afraid to make a mistake,-0.3400
37,Traveling problems,-0.4019,Mental tired,-0.4404
...,...,...,...,...
1202,Communication with MDs,0.0000,Communication with hospital management,0.0000
1203,Not enough employees,0.0000,Workload,0.0000
1218,Weekend coverage,0.0000,Burn out,0.0000
1222,Life balance,0.0000,Stress levels,-0.4215


Mean sentiment: -0.030
Std dev:         0.229


,Q3a,sentiment1,Q3b,sentiment2
0,Improve number,0.4939,Improve pay,0.3612
2,Add reading and projects into basic workload not in leisure time,0.0000,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
28,no,-0.2960,no,-0.2960
35,emotional support,0.5106,financial benefits,0.3818
37,Well programed Work flow,0.2732,Enough personel in IVF lab,0.0000
...,...,...,...,...
1202,More staff,0.0000,More time off to prevent burnout,0.0258
1203,More staff,0.0000,Coordinator to spread cycles efficiently,0.4019
1218,Keeping the # of cases per embryologist at 75,0.0000,Perhaps PRN staffing for weekends.,0.0000
1222,Training more mid level/ junior embryologists,0.0000,Batching/ control of schedule,0.0000


Mean sentiment:  0.142
Std dev:         0.233


,Q4,sentiment
0,Continue,0.0000
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
28,lab director,0.0000
35,lab director,0.0000
37,Retired,0.0000
...,...,...
1202,I’ve already achieved it. Management,0.0000
1203,Train new staff so I can retire,0.0000
1218,Currently have reached my goals-continue good work without errors.,0.6526
1222,Lead managerial embryologist,0.0000


Mean sentiment:  0.199
Std dev:         0.301


,Type of Unit,Q1,Q2,Q3,Q4
0,Academic Public Hospitals,-0.063179,-0.058399,0.143482,0.208301
1,Private Unit,-0.040890,-0.032674,0.133166,0.194092
2,Unit of a Chain,-0.057969,-0.029917,0.142467,0.198997


### Show heatmap

In [415]:
import seaborn as sns
# cm = sns.light_palette("green", as_cmap=True)
# s = cont_results_df.style.background_gradient(cmap='viridis')
s = ToU_results_df.style.background_gradient()
s

,Type of Unit,Q1,Q2,Q3,Q4
0,Academic Public Hospitals,-0.063179,-0.058399,0.143482,0.208301
1,Private Unit,-0.040890,-0.032674,0.133166,0.194092
2,Unit of a Chain,-0.057969,-0.029917,0.142467,0.198997


## 6. Marital status

In [416]:
MS_results = {'Marital status': ['Married or domestic partnership', 'Single, never married', 'Previously Married'],
        'Q1': [0.1, 0.2, 0.3],
        'Q2': [0.1, 0.2, 0.3],
        'Q3': [0.1, 0.2, 0.3],
        'Q4': [0.1, 0.2, 0.3]}
MS_results_df = pd.DataFrame.from_dict(MS_results)
MS_results_df

,Marital status,Q1,Q2,Q3,Q4
0,Married or domestic partnership,0.1,0.1,0.1,0.1
1,"Single, never married",0.2,0.2,0.2,0.2
2,Previously Married,0.3,0.3,0.3,0.3


In [420]:
mdp_df = df[df["Category 4: Marital status"].isin(['Married or domestic partnership'])]
analyse_verbatims(0, mdp_df, MS_results_df)
MS_results_df

,Q1a,sentiment1,Q1b,sentiment2
0,Equality with clinical members,0.0000,Poor pay,-0.5423
2,Working hours and too many responsibilities,0.0000,Salary,0.0000
3,burnout/stress,0.0000,poor management,-0.4767
9,lack of leadership,-0.3182,burnout,0.0000
10,Recognition of our contribution in the ART field,0.0000,Workload,0.0000
...,...,...,...,...
1246,"not getting enough working materials, in ethiopia",0.0000,to get Professional maintenance is difficult in my area,-0.3612
1248,sterility,0.0000,accuracy/prevention of errors,-0.3400
1249,Working lots of weekends and public holidays,0.3818,workload,0.0000
1251,low salaries,-0.2732,weekend and holiday work,0.4019


Mean sentiment: -0.039
Std dev:         0.238


,Q2a,sentiment1,Q2b,sentiment2
0,Shortage of trained staff,-0.2500,Equality,0.0000
2,Access to training,0.0000,Certification,0.0000
3,not enough highly trained staff,0.0000,politics,0.0000
9,managment,0.0000,AI,0.0000
10,Inferior treatment by Clinicians,-0.4019,Workload,0.0000
...,...,...,...,...
1246,maintaining the quality control in the IVF lab,0.0000,"training of good embryology staffs,",0.4404
1248,availability of positions,0.0000,working conditions,0.0000
1249,workload,0.0000,stress,-0.4215
1251,low salaries,-0.2732,weekend and holiday work,0.4019


Mean sentiment: -0.039
Std dev:         0.231


,Q3a,sentiment1,Q3b,sentiment2
0,Improve number,0.4939,Improve pay,0.3612
2,Add reading and projects into basic workload not in leisure time,0.0000,Understand for how many hours a brain can be functional performing lab procedures per day,0.0000
3,better pay,0.3612,better CPD opportunities,0.6705
9,train managers to lead and not manage,0.0000,recognition,0.0000
10,Awareness by Clinic owners,0.0000,Educate us how to successfully get what we are asking for,0.4939
...,...,...,...,...
1246,having good embryology training center,0.4404,support technicians financially to get training abroad,0.4019
1248,Mon to Fri working hours,0.0000,8 hours per day,0.0000
1249,increased wages (should get paid for weekends + public holidays),0.2732,employ more embryologists,0.0000
1251,increase salaries,0.3182,close over christmas,0.0000


Mean sentiment:  0.128
Std dev:         0.226


,Q4,sentiment
0,Continue,0.0000
2,Get my ESHRE certification Publish paper through research Presentations and leading workshops Acquire lab directing skills,0.0000
3,FRCPath,0.0000
9,have a happy team,0.5719
10,This is more of an aspiration - I want my unit to be the best in the country,0.6705
...,...,...
1246,be ART researcher.,0.0000
1248,Building up of newly found laboratory in remote part of country,0.0000
1249,lab director,0.0000
1251,Dont make a big mistake,0.2584


Mean sentiment:  0.185
Std dev:         0.293


,Marital status,Q1,Q2,Q3,Q4
0,Married or domestic partnership,-0.039489,-0.038979,0.128047,0.184573
1,"Single, never married",-0.068982,-0.028170,0.156781,0.222148
2,Previously Married,NaN,NaN,NaN,NaN


In [421]:
snm_df = df[df["Category 4: Marital status"].isin(['Single, never married'])]
analyse_verbatims(1, snm_df, MS_results_df)
MS_results_df

,Q1a,sentiment1,Q1b,sentiment2
1,Bullying by colleagues and managers,-0.5994,"Poorly designed protocols, technical ignorance by colleagues and managers.",-0.3612
4,Recognizion,0.0000,Trust,0.5106
6,General workplace politics,0.0000,Internal Pressure,-0.2960
7,Burnout due to working continously through weekends without breaks,0.0000,Difficulty in taking time off due to understaffed clinics,-0.3400
8,fertilization outcomes are laid on the embryologists,0.0000,pregnancies outcomes are laid on the embryologists,0.0000
...,...,...,...,...
1243,The overtime work (sudden shifts),0.0000,Constant trouble shooting and little room for error,-0.6293
1247,Too many cases per embryologist,0.0000,Not enough off-time,0.0000
1250,Low Salaries,-0.2732,Working weekends,0.0000
1252,Low salaries,-0.2732,Weekend and holiday work,0.4019


Mean sentiment: -0.069
Std dev:         0.247


,Q2a,sentiment1,Q2b,sentiment2
1,Stress of delivering high quality work without sufficient staff.,-0.4215,Presence of narcissistic individuals destroying mental health of colleagues.,-0.5574
4,Handson training,0.0000,Trust,0.5106
6,Undervalued,0.0000,Underpaid,0.0000
7,Lack of qualified embryologists - staff shortages,-0.4404,Underpayment for specialized skills,0.0000
8,overworking with minimum breaks (leave),0.0000,overall outcome of the patient pregnancy rates are laid on the embryologists,0.0000
...,...,...,...,...
1243,Not paid according to the service done,0.0000,The high risk of burnout,-0.2732
1247,Continuous skill development,0.0000,Not enough staff,0.0000
1250,Not closing over December,0.0000,Low Salaries,-0.2732
1252,Low salary,-0.2732,Weekend and holiday work,0.4019


Mean sentiment: -0.028
Std dev:         0.244


,Q3a,sentiment1,Q3b,sentiment2
1,Screening for sociopathic personality traits needs to be introduced. A 45 minute interview is no way to select for entry into the profession.,-0.2960,Clinics must be staffed so that there is a good margin of available workers so that staff are not pressured to work at 110% and then burn out.,0.5523
4,Good laboratory training,0.4404,Troubleshooting,0.1779
6,Flexible hours to reduce stress,-0.2263,Shorter rotations in the Lab,0.0000
7,"Allow for more clinics to be accredited training units, thus increasing accessibility to science graduates and allowing more embryologists into the field",0.2263,"If an embryologist has worked on a weekend, allowing one day off during the week",0.0000
8,improve trust and provide fair team work to embryologist from the clinicians,0.8176,allow reasonable breaks to the embryologist for resting (leave),0.2263
...,...,...,...,...
1243,Schedule stimulation protocols more accordingly,0.0000,Hire people to lessen administrative work,0.0000
1247,More staff to allow more personal time,0.2878,More opportunities to better our knowledge,0.7233
1250,Higher salaries,0.0000,Closing over December,0.0000
1252,Higher salaries,0.0000,Close over Christmas holidays,0.3818


Mean sentiment:  0.157
Std dev:         0.244


,Q4,sentiment
1,To survive until retirement without suffering mental illness or making a catestrophic lab error.,0.2760
4,Be confident in the work i do Academically strong,0.7579
6,"I would like to work as an embryologist internationally, hopefully one day obtain my PhD. I would like to influence up and coming embryologists and have an active role in research within the field of ART",0.8555
7,To be stable and happy while doing a job that I love.,0.8779
8,"to run an independant practice, be a partner and not an employee",0.0000
...,...,...
1243,To practice as a clinical embryologist in a private clinic,0.0000
1247,To be the best embryologist I can be to the patients.,0.6369
1250,To complete my masters and become the best practical embryologist to my abilities.,0.7351
1252,Make babies and make people happy.,0.5719


Mean sentiment:  0.222
Std dev:         0.332


,Marital status,Q1,Q2,Q3,Q4
0,Married or domestic partnership,-0.039489,-0.038979,0.128047,0.184573
1,"Single, never married",-0.068982,-0.028170,0.156781,0.222148
2,Previously Married,NaN,NaN,NaN,NaN


In [422]:
pm_df = df[df["Category 4: Marital status"].isin(['Previously Married'])]
analyse_verbatims(2, pm_df, MS_results_df)
MS_results_df

,Q1a,sentiment1,Q1b,sentiment2
5,Stress created by no embryos/ less embryos/ poor quality embryos,-0.7024,No respect,0.2263
14,Time spent at work,0.0000,Money for new quipment,0.0000
18,"hard cases; bad sperm, fragile oocytes",-0.5994,verification,0.0000
61,Excessive overtime,0.0000,Career progression,0.0000
66,high workload,0.0000,stress,-0.4215
...,...,...,...,...
1203,Not enough employees,0.0000,Time off work,0.0000
1210,Long hours,0.0000,Poor equipment,-0.4767
1215,Weekend rota,0.0000,Failed fertilisation,-0.5106
1232,Burnout,0.0000,Improper staffing,0.0000


Mean sentiment: -0.051
Std dev:         0.259


,Q2a,sentiment1,Q2b,sentiment2
5,Gossips of others which disturbs other Embryologist work,-0.6369,No respect,0.2263
14,Low salary,-0.2732,Stress,-0.4215
18,long working hours,0.0000,stress,-0.4215
61,Overworked,0.0000,Underpaid,0.0000
66,clinician dominance,0.2023,ethical problems,0.1531
...,...,...,...,...
1203,Not enough employees,0.0000,Workload,0.0000
1210,Long hours,0.0000,Poor pay,-0.5423
1215,Recognition from management,0.0000,Managing patient’s expectations,0.0000
1232,Burnout,0.0000,Improper ataffing,0.0000


Mean sentiment: -0.045
Std dev:         0.245


,Q3a,sentiment1,Q3b,sentiment2
5,Reporting Manager should understand the situation,0.0,Stress free atmosphere,0.1280
14,More staff,0.0,more off time,0.0000
18,organisation,0.0,"good comminication with whole team; doctors, nurses, lab technicians",0.4404
61,More embryologists,0.0,Better pay,0.3612
66,less overtime,0.0,more meetings with clinicians,0.0000
...,...,...,...,...
1203,More staff,0.0,Coordinator to spread cycles efficiently,0.4019
1210,Hire more staff,0.0,Higher wates,0.0000
1215,Sufficient staffing levels,0.0,Team building supports,0.3612
1232,Setting realistic staffing ratios,0.0,Recognizing that embryologists are not just Lab Techs,0.0000


Mean sentiment:  0.149
Std dev:         0.234


,Q4,sentiment
5,To be Good Embryologist by providing maximum effort and good professor by teaching others,0.7003
14,Best success rate to promote our clinic,0.8885
18,create a routine for personilized treatment for patients,0.2732
61,Be the best I can at the job I do to provide patients with best possible care,0.9118
66,-,0.0000
...,...,...
1203,Train new staff so I can retire,0.0000
1210,To retire,0.0000
1215,Maintain and improve standards in the field.,0.4404
1232,To retire,0.0000


Mean sentiment:  0.236
Std dev:         0.326


,Marital status,Q1,Q2,Q3,Q4
0,Married or domestic partnership,-0.039489,-0.038979,0.128047,0.184573
1,"Single, never married",-0.068982,-0.028170,0.156781,0.222148
2,Previously Married,-0.051153,-0.044538,0.148555,0.236433


### Show heatmap

In [423]:
import seaborn as sns
# cm = sns.light_palette("green", as_cmap=True)
# s = cont_results_df.style.background_gradient(cmap='viridis')
s = MS_results_df.style.background_gradient()
s

,Marital status,Q1,Q2,Q3,Q4
0,Married or domestic partnership,-0.039489,-0.038979,0.128047,0.184573
1,"Single, never married",-0.068982,-0.028170,0.156781,0.222148
2,Previously Married,-0.051153,-0.044538,0.148555,0.236433
